In [ ]:
#---- if not installed ----

#pip install --upgrade PyQt5 PyQt5-sip
#pip install PyQtWebEngine
#pip install --upgrade PyQt5 PyQt5-sip PyQtWebEngine

In [1]:
#!!!! PLEASE CHOOSE YOUR RESOLTUION/WINDOW SIZE:
current_width = 1920   # CHANGE THIS
current_height = 1080  # CHANGE THIS

#!! DONT CHANGE THESE!
base_width = 1920   # DONT CHANGE
base_height = 1080  # DONT CHANGE

In [2]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import re
import tempfile
import time
import gc
from math import pi
import plotly.express as px
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
    NavigationToolbar2QT as NavigationToolbar,
)
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import *
from PyQt5.QtWebEngineWidgets import QWebEngineView, QWebEngineSettings
from PyQt5.uic import loadUi
from PyQt5.QtCore import *
from PyQt5.QtCore import QThread, pyqtSignal
from PyQt5.QtGui import QPixmap
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist

In [3]:
# --------------------------------------------------------------------------------------------------------------------------
# Defining global variables (needed later)


metric_features = ['vendor_count', 'product_count', 'is_chain',
       'first_order', 'last_order', 'lifetime_days', 'total_expenses',
       'avg_per_product', 'avg_per_order', 'avg_order_size',
       'culinary_variety', 'chain_preference', 'loyalty_to_venders', 'customer_age_group']

prefBasedFeatures = ['culinary_variety', 'loyalty_to_venders', 'chain_preference', 'is_chain']
purchaseBasedFeatures = ['vendor_count', 'product_count', 'total_expenses', 'avg_per_product', 'avg_per_order', 'avg_order_size']
ageTimeBasedFeatures = ['first_order', 'last_order', 'lifetime_days', 'customer_age_group']
currentFeatures = metric_features.copy()   # will be changed later based on current choice for clustering info

#for scaling:
metric_features_scale = ['lifetime_days', 'total_expenses', 'avg_per_product', 'avg_per_order', 'avg_order_size',
       'culinary_variety', 'chain_preference', 'loyalty_to_venders', 'customer_age_group']
group1 = ['first_order', 'last_order']
group2 = ['is_chain', 'vendor_count', 'product_count']

#values for later calculation of final values
last_order = 0
first_order = 0
lifetime_days = 0
total_expenses = 0
avg_per_product = 0
avg_per_order = 0
avg_order_size = 0
culinary_variety = 0
chain_preference = 0
loyalty_to_venders = 0
customer_age_group = 0
vendor_count = 0
product_count = 0
is_chain = 0
OrdersPlaced = 0


#saved labels, scalers, clusters and df
labelsFinal = np.loadtxt("../dataInterface/meta_labels_Final6.csv", delimiter=",", dtype=int)
labels15 = np.loadtxt("../dataInterface/labels_1_5.csv", delimiter=",", dtype=int)
labels25 = np.loadtxt("../dataInterface/labels_2_5.csv", delimiter=",", dtype=int)
labels35 = np.loadtxt("../dataInterface/labels_3_5.csv", delimiter=",", dtype=int)
labels = np.copy(labelsFinal)

clusters = joblib.load('../dataInterface/FINAL_clusters_hc6.pkl')

scaler1 = joblib.load('../dataInterface/scalerGroupBasic.pkl')
scaler2 = joblib.load('../dataInterface/scalerGroup1.pkl')
scaler3 = joblib.load('../dataInterface/scalerGroup2.pkl')

cluster_ids_set = list(map(int, sorted(set(labels))))  # to get teh unique values of labels (0-4 for 5 clusterings and 0-5 for 6 clustering)

df = pd.read_csv("../dataInterface/DfScaling_Updatet.csv")
df_concat = pd.concat([df[metric_features],pd.Series(labels, name='labels', index=df.index)],axis=1)


# global variables needed for functionality
is_growing = True
lastGraph2 = 0
lastGraph3 = 0
lastGraph4 = 0
lastGraphCheck3 = 0
lastGraphPage2 = 0
lastGraphPage3 = 0
lastGraphPage4 = 0
ChangeCombo = True
assignedCluster2 = 17

scale_factor_graph = min(current_width / base_width, current_height / base_height)
graph_size = int(8*scale_factor_graph)

#dictionaries used later
warn_values = {
    "TT1_input": False, "TT2_input": False, "TT3_input": False, "TT4_input": False, "TT5_input": False, "TT6_slider": False, "TT9_input": False, # if there is a warning for unsual values
}

confirmed_status = {
    "TT1_input": False, "TT2_input": False, "TT3_input": False, "TT4_input": False, "TT5_input": False, "TT6_slider_label": False,
    "TT7_slider_label": False, "TT8_combo": False,"TT9_input": False,
}

In [4]:
# ----------------------------------------------------------------------------------------------------------------------------
# INFO: similar concepts are only explained once, so not at every button there are comments, just at the first one for example
# ----------------------------------------------------------------------------------------------------------------------------

class ClusteringThread(QThread):   # Thread to recalculate clusters with new point (thread used so does not impede with main window)
    clustering_finished = pyqtSignal()  # define signal for when thread is finished
    progress = pyqtSignal(str)  # define signal used for changing text of loading label depending on current state

    def __init__(self, data, n_clusters, group):  # get argument values
        super().__init__()
        self.data = data   #dataframe
        self.n_clusters = n_clusters   # number of clusters
        self.group = group   # group, 0 for recalculating final clustering, 1 for recalculating only one of the perspectives

    def run(self):
        global assignedCluster2  #for final label

        if(self.group == 1):   # if just recalculate one of cluster perspectives
            self.progress.emit("...loading...")   # text of loading label
            try:
                clustering = AgglomerativeClustering(linkage='ward', metric='euclidean', n_clusters=self.n_clusters)   # clustering with n_clusters

                temp_labels = clustering.fit_predict(self.data[currentFeatures])  # fit and predict for current features from df from argument

                temp_labels_series = pd.Series(temp_labels, name='labels', index=self.data.index)

                df_concat_temp = pd.concat([self.data, temp_labels_series], axis=1)   # concat dataframe and new labels
                assignedCluster2 = df_concat_temp.loc[df_concat_temp.index[-1], 'labels']  # assigned cluster = label from last row of df

                print("assignedcluster2", assignedCluster2)
            except Exception as e:
                print("ExceptClusterTryGroups1-3", e)

        elif self.group == 0:   # if recalculate final clustering
            self.progress.emit("...clustering group1...")   # text of loading label
            try:  # first the clustering for all three persepectives has to be redone, starting with the first one
                clust1 = AgglomerativeClustering(linkage='ward', metric='euclidean', n_clusters=5)
                c1labels = clust1.fit_predict(self.data[prefBasedFeatures])
                c1labels_series = pd.Series(c1labels, name='labels1', index=self.data.index)
                df_concat_c1 = pd.concat([self.data[prefBasedFeatures], c1labels_series], axis=1)
            except Exception as e:
                print("err1", e)
            self.progress.emit("...clustering group2...")
            try:   # second perspective
                clust2 = AgglomerativeClustering(linkage='ward', metric='euclidean', n_clusters=5)
                c2labels = clust2.fit_predict(self.data[purchaseBasedFeatures])
                c2labels_series = pd.Series(c2labels, name='labels2', index=self.data.index)
                df_concat_c2 = pd.concat([self.data[purchaseBasedFeatures], c2labels_series], axis=1)
            except Exception as e:
                print("err2", e)
            self.progress.emit("...clustering group3...")
            try:   # third perspective
                clust3 = AgglomerativeClustering(linkage='ward', metric='euclidean', n_clusters=5)
                c3labels = clust3.fit_predict(self.data[ageTimeBasedFeatures])
                c3labels_series = pd.Series(c3labels, name='labels3', index=self.data.index)
                df_concat_c3 = pd.concat([self.data[ageTimeBasedFeatures], c3labels_series], axis=1)
            except Exception as e:
                print("err3", e)

            try:   # add all 3 new labels to new df
                combined_df = pd.concat([df_concat_c1, df_concat_c2, df_concat_c3], axis=1)
                features = combined_df[['labels1', 'labels2', 'labels3']].values
                weights = [1.5, 1.5, 1.0]
                features_weighted = features * weights  # add weights to these labels
            except Exception as e:
                print("err4", e)
            self.progress.emit("...calculating distance matrix...")   # text of loading label
            try:
                euclidean_distances = pdist(features_weighted, metric='euclidean')  # calculate distance between weighted labels
                del features_weighted  # delete some not anymore used variables from memory, since matrix needs a lot of memory
                del combined_df
                del features
                del c1labels
                del c2labels
                del c3labels
                del df_concat_c1
                del df_concat_c2
                del df_concat_c3
                distance_matrix = squareform(euclidean_distances)  # create distance matrix

            except Exception as e:
                print("err5", e)
            self.progress.emit("...calculating final cluster...")
            try:
                gc.collect()   # for more memory space
                hclust_final = AgglomerativeClustering(   # clustering for final clustering
                    linkage='average',
                    metric='precomputed',
                    n_clusters=6
                )
                meta_labels = hclust_final.fit_predict(distance_matrix)  # fit and predict labels for distance matrix
                del distance_matrix  # delete distance matrix again from memory, because very big
            except Exception as e:
                print("err6", e)
            self.progress.emit("...loading...")
            try:
                final_labels_series = pd.Series(meta_labels, name='metalabels', index=self.data.index)   # create labels series
                df_concat_final = pd.concat([self.data[metric_features], final_labels_series], axis=1)
            except Exception as e:
                print("err7", e)
            try:
                assignedCluster2 = df_concat_final.loc[df_concat_final.index[-1], 'metalabels'] # get last row from new df, which has the final label
            except Exception as e:
                print("ultErr", e)
            print("assig2", assignedCluster2)

        self.clustering_finished.emit()  # emit signal because clustering is finished


class Ui_MainWindow(object):   # MainWindow

    def apply_stylesheet(self, widget, border_color="white", focus_border_color="blue", border_px="2px",border_status="solid"):  # method to change stylesheet
        widget_name = widget.objectName()
        current_stylesheet = widget.styleSheet()  # get current style sheet

        new_stylesheet = current_stylesheet

        new_stylesheet = re.sub(  # change border
            r'border:.*?;',
            f'border: {border_px} {border_status} {border_color};',
            new_stylesheet,
            flags=re.DOTALL
        )

        if isinstance(widget, QComboBox):  # if combobox
            scale_factor = min(current_width / base_width,
                               current_height / base_height)  # scale fontsize based on windowsize
            font_size = int(16 * scale_factor)
            new_stylesheet = re.sub(
                r'QComboBox \{.*?border:.*?;',
                f'''QComboBox {{ border: {border_px} {border_status} {border_color};
                                 color: black;
                                 background-color: white;
                                 font-size: {font_size}px;
                }}''',
                new_stylesheet,
                flags=re.DOTALL
            )
            if "QAbstractItemView" not in new_stylesheet:   # add AbstractItemView
                new_stylesheet += f"""
                        QComboBox QAbstractItemView {{
                            border: {border_px} {border_status} {border_color};
                            background-color: {background_color};
                            font-size: {font_size}px;
                        }}
                        """

        if isinstance(widget, QSlider):   # if slider
            scale_factor = min(current_width / base_width, current_height / base_height)
            font_size = int(16 * scale_factor)

            new_stylesheet += f"""
                        QSlider {{
                            border: {border_px} {border_status} {border_color};
                            font-size: {font_size}px;  /* Schriftgröße setzen */
                        }}
                        QSlider::handle {{
                            border: {border_px} {border_status} {border_color};  /* Rand für den Schieberegler */
                            background: {border_color};  /* Hintergrundfarbe des Schiebereglers */
                        }}
                    """

        widget.setStyleSheet(new_stylesheet)


    def set_relative_geometry_3(self, widget, x, y, width, height,
                                base_width, base_height, current_width, current_height,
                                fontsize=16, paddingsize=5):  # make geometry relative to window size and add fontSize/Padding

        relative_x = (x / base_width) * current_width    # calculate relative values for geometry (base values are 1920x1080)
        relative_y = (y / base_height) * current_height
        relative_width = (width / base_width) * current_width
        relative_height = (height / base_height) * current_height

        widget.setGeometry(int(relative_x), int(relative_y), int(relative_width), int(relative_height))  # set geometry

        scale_factor = min(current_width / base_width, current_height / base_height)  # calculate scale factors used for fontsize and padding
        new_font_size = int(fontsize * scale_factor)
        new_padding = int(paddingsize * scale_factor)
        current_stylesheet = widget.styleSheet()
        dynamic_stylesheet = ""
        if isinstance(widget, QComboBox):
            dynamic_stylesheet = f"""
                QComboBox {{
                    font-size: {new_font_size}px;
                    padding: {new_padding}px;
                }}
                QComboBox::drop-down {{
                    font-size: {new_font_size}px;
                }}
                QComboBox QAbstractItemView {{
                    font-size: {new_font_size}px;
                }}
                """


        elif isinstance(widget, QSlider):
            orientation = "horizontal" if widget.orientation() == Qt.Horizontal else "vertical"
            groove_height = int(8 * scale_factor)
            groove_width = int(218 * scale_factor)
            handle_size = int(18 * scale_factor)
            dynamic_stylesheet = f"""
                    QSlider {{
                        font-size: {new_font_size}px;
                        padding: {new_padding}px;
                        border: none;
                        height: {groove_height}px;  /* Dynamische Höhe des Sliders */
                        width: {groove_width}px;
                    }}
                    QSlider::groove:horizontal {{
                        background: white;
                        height: {groove_height}px;
                        width: {groove_width}px;
                        border-radius: 4px;
                    }}
                    QSlider::handle::{orientation} {{
                        background: #333;
                        border: 2px solid #555;
                        width: {handle_size}px;  /* Skaliere die Breite des Handgriffs */
                        height: {handle_size}px;  /* Skaliere die Höhe des Handgriffs */
                        margin: -{handle_size // 2}px 0;  /* Platziere den Handle außerhalb des Sliders */
                        border-radius: {handle_size // 2}px;
                    }}
                """

        elif isinstance(widget, QtWidgets.QLabel):
            dynamic_stylesheet = f"""
                    QLabel {{
                        font-size: {new_font_size}px;
                        padding: {new_padding}px;
                    }}
                """
        elif isinstance(widget, QtWidgets.QLineEdit):
            dynamic_stylesheet = f"""
                        QLineEdit {{
                            font-size: {new_font_size}px;
                            padding: {new_padding}px;
                        }}
                    """
        elif isinstance(widget, QtWidgets.QPushButton):
            dynamic_stylesheet = f"""
                    QPushButton {{
                        font-size: {new_font_size}px;
                        padding: {new_padding}px;
                    }}
                """
        widget.setStyleSheet(current_stylesheet + dynamic_stylesheet)   # add new stylesheet to current
        widget.adjustSize()

    def set_relative_geometry_2(self, widget, x, y, width, height,
                                base_width, base_height, current_width, current_height):  # change relative geometry only, not fontsize/padding

        relative_x = (x / base_width) * current_width
        relative_y = (y / base_height) * current_height
        relative_width = (width / base_width) * current_width
        relative_height = (height / base_height) * current_height

        widget.setGeometry(int(relative_x), int(relative_y), int(relative_width), int(relative_height))


    def on_combobox_change(self, combo_name,error_label_name):  # method for when there is a change in combobox, to reset its border and unconfirm it
        global confirmed_status
        combo_box = getattr(self, combo_name)
        error_label = getattr(self, error_label_name)
        scale_factor = min(current_width / base_width,
                           current_height / base_height)  # scale fontsize and padding based on window size
        new_font_size = int(16 * scale_factor)
        new_padding = int(5 * scale_factor)
        combo_box.setStyleSheet(f"""
                                QComboBox {{
                                        background-color: white;
                                        color: black;
                                        border: 2px solid white;
                                        font-size: {new_font_size}px;
                                        padding: {new_padding}px;
                                        border-radius: 10px;
                                    }}
                                QComboBox:focus {{
                                    border: 2px solid blue;
                                }}
                                QComboBox:editable:on {{
                                    border: 2px solid blue;
                                }}
                                QComboBox:!editable:on {{
                                    border: 2px solid blue;
                                }}
                                QComboBox QAbstractItemView {{
                                    background-color: white;
                                    font-size: {new_font_size}px;
                                    color: black;
                                    border: 2px solid white;
                                    selection-background-color: lightblue;
                                    selection-color: black;
                                }}
                                            """)
        error_label.setVisible(False)
        confirmed_status[combo_name] = False

    def on_combobox_change_2(self, combo_name):   # method for changes on combobox to call again the last called graph, but with new value
        # also used for any changes in the combobox on top of screen which lets decide between the clustering used
        combo_box = getattr(self, combo_name)
        global lastGraph2
        global lastGraph3
        global lastGraph4
        global labels
        global df_concat
        global currentFeatures
        global cluster_ids_set
        global ChangeCombo
        if "MP_combo" in combo_name:   # if change was in combobox on top of screen which lets decide between the clusterings used
            self.graph_label3.clear()   # make labels with graphs empty (showing no more graph)
            self.graph_label4.clear()
            if self.MP_combo.currentIndex() == 0:  # if index is 0 (final clustering is selected)
                labels = np.copy(labelsFinal)   # get the labels, add them to df
                df_concat = pd.concat([df[metric_features], pd.Series(labels, name='labels', index=df.index)], axis=1)
                cluster_ids_set = list(map(int, sorted(set(labels))))
                currentFeatures = metric_features.copy()   # use metric features as current features (because this uses all features)
                ChangeCombo = False   # there will be changes made to comboboxes by adding new items, but these should not recall the current
                                      # change_on_comebobox_2 method
                if self.P2_combo.count() < 5:    # if there are only 5 elements, add a 6th one (because final clustering has 6 clusters, others have 5)
                    self.P2_combo.addItem("5")
                self.C5_checkbox.setVisible(True)  # make the 6th checkbox visible, same reason
                self.P3_combo.clear()  # clear p3 and p4 combobox and add choose plus current features ( in order to only have current features to choice)
                self.P3_combo.addItems(["--choose--"] + currentFeatures)
                self.P3_combo.view().setMinimumHeight(300)
                self.P4_combo.clear()
                self.P4_combo.addItems(["--choose--"] + currentFeatures)
                self.P4_combo.view().setMinimumHeight(300)
                ChangeCombo = True  # changes made in comboboxes now should again call this current method on_combobox_change_2
                if lastGraphPage2 != 0:   # if last plotted graph on page 2 is not 0 (0 when not yet plotted on this page), then call method to plot latest plot again
                    method_name2 = f"plot_2_{lastGraphPage2}"
                    method2 = getattr(self, method_name2, None)
                    method2()
                if lastGraphPage3 != 0:
                    method_name3 = f"plot_3_{lastGraphPage3}"
                    method3 = getattr(self, method_name3, None)
                    method3()
                if lastGraphPage4 != 0:
                    method_name4 = f"plot_4_{lastGraphPage4}"
                    method4 = getattr(self, method_name4, None)
                    method4()
                self.resetAfterNewCluster()  # for the addNewPointToCluster page resetting the page by making all error labels invisible etc
                self.labelInfoCluster2.setText(   # change the info texts for current clustering)
                    "Cluster 0: regular but more spending customers\n"
                    "Cluster 1: largest group of regular customers\n"
                    "Cluster 2: frequent orders but low loyality\n"
                    "Cluster 3: high frequency, small orders\n"
                    "Cluster 4: best spending customers\n"
                    "Cluster 5: least spending customers")
                self.labelInfoCluster2.adjustSize()
                self.labelInfoCluster3.setText(
                    "Cluster 0: regular but more spending customers\n"
                    "Cluster 1: largest group of regular customers\n"
                    "Cluster 2: frequent orders but low loyality\n"
                    "Cluster 3: high frequency, small orders\n"
                    "Cluster 4: best spending customers\n"
                    "Cluster 5: least spending customers")
                self.labelInfoCluster3.adjustSize()
                self.labelInfoCluster4.setText(
                    "Cluster 0: regular but more spending customers\n"
                    "Cluster 1: largest group of regular customers\n"
                    "Cluster 2: frequent orders but low loyality\n"
                    "Cluster 3: high frequency, small orders\n"
                    "Cluster 4: best spending customers\n"
                    "Cluster 5: least spending customers")
                self.labelInfoCluster4.adjustSize()
                self.calc_info_label.setText(
                    "INFO: Other values will be calculated based on these:\n\nlifetime_days = last_order - first_order\n"
                    "avg_per_product = total_expenses / product_count\navg_per_order = total_expenses / OrdersPlaced\n"
                    "avg_order_size = product_count / OrdersPlaced\nchain_preference = is_chain / OrdersPlaced\n"
                    "loyalty_to_venders = vendor_count / OrdersPlaced")

                # reorganize the buttons on the last page, depending on which are visible for current clustering
                self.set_relative_geometry_2(self.TT3_label, 650, 150, 130, 30, base_width, base_height, current_width,current_height)
                self.set_relative_geometry_2(self.TT3_input, 650, 200, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT3_Confirm, 650, 250, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT3_err, 650, 300, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT3_warn, 650, 300, 180, 40, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_label, 950, 150, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_input, 950, 200, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_Confirm, 950, 250, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_err, 950, 300, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_warn, 950, 300, 180, 40, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT7_label, 580, 650, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT7_input_slider, 530, 700, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT7_slider_label, 580, 735, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT7_Confirm, 580, 785, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_label, 50, 150, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_input, 50, 200, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_Confirm, 50, 250, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_err, 50, 300, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_warn, 50, 300, 180, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT2_label, 350, 150, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT2_input, 350, 200, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT2_Confirm, 350, 250, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT2_err, 350, 300, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT2_warn, 350, 300, 180, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT6_label, 230, 650, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT6_input_slider, 180, 700, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT6_slider_label, 230, 735, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT6_Confirm, 230, 785, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT6_warn, 180, 850, 180, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT4_label, 350, 400, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT4_input, 350, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT4_Confirm, 350, 500, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT4_err, 350, 550, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT4_warn, 350, 550, 180, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT5_label, 650, 400, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT5_input, 650, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT5_Confirm, 650, 500, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT5_err, 650, 550, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT5_warn, 650, 550, 180, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT8_label, 50, 400, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT8_combo, 50, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT8_Confirm, 50, 500, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT8_err, 50, 550, 180, 40, base_width, base_height,
                                             current_width, current_height)

                self.TT1_input.setVisible(True)   # making buttons visible or invisible depending on current clustering
                self.TT1_label.setVisible(True)
                self.TT1_Confirm.setVisible(True)
                self.TT2_input.setVisible(True)
                self.TT2_label.setVisible(True)
                self.TT2_Confirm.setVisible(True)
                self.TT3_input.setVisible(True)
                self.TT3_label.setVisible(True)
                self.TT3_Confirm.setVisible(True)
                self.TT4_input.setVisible(True)
                self.TT4_label.setVisible(True)
                self.TT4_Confirm.setVisible(True)
                self.TT5_input.setVisible(True)
                self.TT5_label.setVisible(True)
                self.TT5_Confirm.setVisible(True)
                self.TT6_slider_label.setVisible(True)
                self.TT6_input_slider.setVisible(True)
                self.TT6_label.setVisible(True)
                self.TT6_Confirm.setVisible(True)
                self.TT7_slider_label.setVisible(True)
                self.TT7_input_slider.setVisible(True)
                self.TT7_label.setVisible(True)
                self.TT7_Confirm.setVisible(True)
                self.TT8_combo.setVisible(True)
                self.TT8_label.setVisible(True)
                self.TT8_Confirm.setVisible(True)
                self.TT9_input.setVisible(True)
                self.TT9_label.setVisible(True)
                self.TT9_Confirm.setVisible(True)
            if self.MP_combo.currentIndex() == 1:   # if clustering choosen in combobox on top is the second one (PreferenceBased). similar functionality
                labels = np.copy(labels15)
                df_concat = pd.concat([df[prefBasedFeatures], pd.Series(labels, name='labels', index=df.index)], axis=1)
                cluster_ids_set = list(map(int, sorted(set(labels))))
                currentFeatures = prefBasedFeatures.copy()
                ChangeCombo = False
                if self.P2_combo.count() > 5:
                    self.P2_combo.removeItem(5)
                self.C5_checkbox.setVisible(False)
                self.C5_checkbox.setChecked(False)
                self.P3_combo.clear()
                self.P3_combo.addItems(["--choose--"] + currentFeatures)
                self.P3_combo.view().setMinimumHeight(100)
                self.P4_combo.clear()
                self.P4_combo.addItems(["--choose--"] + currentFeatures)
                self.P4_combo.view().setMinimumHeight(100)
                ChangeCombo = True
                if lastGraphPage2 != 0:
                    method_name2 = f"plot_2_{lastGraphPage2}"
                    method2 = getattr(self, method_name2, None)
                    method2()
                if lastGraphPage3 != 0:
                    method_name3 = f"plot_3_{lastGraphPage3}"
                    method3 = getattr(self, method_name3, None)
                    method3()
                if lastGraphPage4 != 0:
                    method_name4 = f"plot_4_{lastGraphPage4}"
                    method4 = getattr(self, method_name4, None)
                    method4()
                self.resetAfterNewCluster()
                self.labelInfoCluster2.setText(
                    "Cluster 0: regular with higher loyality and lower Chain-orders\n"
                    "Cluster 1: high culinary variety and Chain-orders\n"
                    "Cluster 2: low Chain-Orders, Chain-Preference and loyality\n"
                    "Cluster 3: regular with low loyality\n"
                    "Cluster 4: low Chain-Orders and preference but higher loyality")
                self.labelInfoCluster2.adjustSize()
                self.labelInfoCluster3.setText(
                    "Cluster 0: regular with higher loyality and lower Chain-orders\n"
                    "Cluster 1: high culinary variety and Chain-orders\n"
                    "Cluster 2: low Chain-Orders, Chain-Preference and loyality\n"
                    "Cluster 3: regular with low loyality\n"
                    "Cluster 4: low Chain-Orders and preference but higher loyality")
                self.labelInfoCluster3.adjustSize()
                self.labelInfoCluster4.setText(
                    "Cluster 0: regular with higher loyality and lower Chain-orders\n"
                    "Cluster 1: high culinary variety and Chain-orders\n"
                    "Cluster 2: low Chain-Orders, Chain-Preference and loyality\n"
                    "Cluster 3: regular with low loyality\n"
                    "Cluster 4: low Chain-Orders and preference but higher loyality")
                self.labelInfoCluster4.adjustSize()
                self.calc_info_label.setText(
                    "INFO: Other values will be calculated based on these:\n\nchain_preference = is_chain / OrdersPlaced\n"
                    "loyalty_to_venders = vendor_count / OrdersPlaced")

                self.set_relative_geometry_2(self.TT3_label, 500, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT3_input, 500, 400, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT3_Confirm, 500, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT3_err, 500, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT3_warn, 500, 500, 380, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT9_label, 750, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_input, 750, 400, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_Confirm, 750, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_err, 750, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_warn, 750, 500, 380, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT1_label, 250, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_input, 250, 400, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_Confirm, 250, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_err, 250, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_warn, 250, 500, 380, 40, base_width, base_height,
                                             current_width, current_height)


                self.set_relative_geometry_2(self.TT7_label, 500, 570, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT7_input_slider, 450, 600, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT7_slider_label, 500, 635, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT7_Confirm, 500, 685, 130, 30, base_width, base_height,
                                             current_width, current_height)

                self.TT1_input.setVisible(True)
                self.TT1_label.setVisible(True)
                self.TT1_Confirm.setVisible(True)
                self.TT2_input.setVisible(False)
                self.TT2_label.setVisible(False)
                self.TT2_Confirm.setVisible(False)
                self.TT3_input.setVisible(True)
                self.TT3_label.setVisible(True)
                self.TT3_Confirm.setVisible(True)
                self.TT4_input.setVisible(False)
                self.TT4_label.setVisible(False)
                self.TT4_Confirm.setVisible(False)
                self.TT5_input.setVisible(False)
                self.TT5_label.setVisible(False)
                self.TT5_Confirm.setVisible(False)
                self.TT6_slider_label.setVisible(False)
                self.TT6_input_slider.setVisible(False)
                self.TT6_label.setVisible(False)
                self.TT6_Confirm.setVisible(False)
                self.TT7_slider_label.setVisible(True)
                self.TT7_input_slider.setVisible(True)
                self.TT7_label.setVisible(True)
                self.TT7_Confirm.setVisible(True)
                self.TT8_combo.setVisible(False)
                self.TT8_label.setVisible(False)
                self.TT8_Confirm.setVisible(False)
                self.TT9_input.setVisible(True)
                self.TT9_label.setVisible(True)
                self.TT9_Confirm.setVisible(True)
            if self.MP_combo.currentIndex() == 2:   # if 3rd is choosen in combobox (Purchase perspective cluster
                labels = np.copy(labels25)
                df_concat = pd.concat([df[purchaseBasedFeatures], pd.Series(labels, name='labels', index=df.index)], axis=1)
                cluster_ids_set = list(map(int, sorted(set(labels))))
                currentFeatures = purchaseBasedFeatures.copy()
                ChangeCombo = False
                if self.P2_combo.count() > 5:
                    self.P2_combo.removeItem(5)
                self.C5_checkbox.setVisible(False)
                self.C5_checkbox.setChecked(False)
                self.P3_combo.clear()
                self.P3_combo.addItems(["--choose--"] + currentFeatures)
                self.P3_combo.view().setMinimumHeight(120)
                self.P4_combo.clear()
                self.P4_combo.addItems(["--choose--"] + currentFeatures)
                self.P4_combo.view().setMinimumHeight(120)
                ChangeCombo = True
                if lastGraphPage2 != 0:
                    method_name2 = f"plot_2_{lastGraphPage2}"
                    method2 = getattr(self, method_name2, None)
                    method2()
                if lastGraphPage3 != 0:
                    method_name3 = f"plot_3_{lastGraphPage3}"
                    method3 = getattr(self, method_name3, None)
                    method3()
                if lastGraphPage4 != 0:
                    method_name4 = f"plot_4_{lastGraphPage4}"
                    method4 = getattr(self, method_name4, None)
                    method4()
                self.resetAfterNewCluster()
                self.labelInfoCluster2.setText(
                    "Cluster 0: regular orders and expenses, but high averages p. Product/Order and Order size \n"
                    "Cluster 1: low avg Ordersizes, also low Product-/Vendorcount and total expenses\n"
                    "Cluster 2: high Product-/Vendorcount, low avgs -> more smaller orders\n"
                    "Cluster 3: very low avgs and total expenses, regular Product-/Vendorcount\n"
                    "Cluster 4: High Product-/Vendorcount, also high avgs -> frequent and big orders")
                self.labelInfoCluster2.adjustSize()
                self.labelInfoCluster3.setText(
                    "Cluster 0: regular orders and expenses, but high averages p. Product/Order and Order size \n"
                    "Cluster 1: low avg Ordersizes, also low Product-/Vendorcount and total expenses\n"
                    "Cluster 2: high Product-/Vendorcount, low avgs -> more smaller orders\n"
                    "Cluster 3: very low avgs and total expenses, regular Product-/Vendorcount\n"
                    "Cluster 4: High Product-/Vendorcount, also high avgs -> frequent and big orders")
                self.labelInfoCluster3.adjustSize()
                self.labelInfoCluster4.setText(
                    "Cluster 0: regular orders and expenses, but high averages p. Product/Order and Order size \n"
                    "Cluster 1: low avg Ordersizes, also low Product-/Vendorcount and total expenses\n"
                    "Cluster 2: high Product-/Vendorcount, low avgs -> more smaller orders\n"
                    "Cluster 3: very low avgs and total expenses, regular Product-/Vendorcount\n"
                    "Cluster 4: High Product-/Vendorcount, also high avgs -> frequent and big orders")
                self.labelInfoCluster4.adjustSize()
                self.calc_info_label.setText(
                    "INFO: Other values will be calculated based on these:\n\n"
                    "avg_per_product = total_expenses / product_count\navg_per_order = total_expenses / OrdersPlaced\n"
                    "avg_order_size = product_count / OrdersPlaced")

                self.set_relative_geometry_2(self.TT1_label, 250, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_input, 250, 400, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_Confirm, 250, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_err, 250, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT1_warn, 250, 500, 380, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT2_label, 500, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT2_input, 500, 400, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT2_Confirm, 500, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT2_err, 500, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT2_warn, 500, 500, 380, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT6_label, 500, 590, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT6_input_slider, 450, 620, 230, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT6_slider_label, 500, 655, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT6_Confirm, 500, 705, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT6_warn, 470, 750, 180, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT9_label, 750, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_input, 750, 400, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_Confirm, 750, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_err, 750, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT9_warn, 750, 500, 380, 40, base_width, base_height,
                                             current_width, current_height)

                self.TT1_input.setVisible(True)
                self.TT1_label.setVisible(True)
                self.TT1_Confirm.setVisible(True)
                self.TT2_input.setVisible(True)
                self.TT2_label.setVisible(True)
                self.TT2_Confirm.setVisible(True)
                self.TT3_input.setVisible(False)
                self.TT3_label.setVisible(False)
                self.TT3_Confirm.setVisible(False)
                self.TT4_input.setVisible(False)
                self.TT4_label.setVisible(False)
                self.TT4_Confirm.setVisible(False)
                self.TT5_input.setVisible(False)
                self.TT5_label.setVisible(False)
                self.TT5_Confirm.setVisible(False)
                self.TT6_slider_label.setVisible(True)
                self.TT6_input_slider.setVisible(True)
                self.TT6_label.setVisible(True)
                self.TT6_Confirm.setVisible(True)
                self.TT7_slider_label.setVisible(False)
                self.TT7_input_slider.setVisible(False)
                self.TT7_label.setVisible(False)
                self.TT7_Confirm.setVisible(False)
                self.TT8_combo.setVisible(False)
                self.TT8_label.setVisible(False)
                self.TT8_Confirm.setVisible(False)
                self.TT9_input.setVisible(True)
                self.TT9_label.setVisible(True)
                self.TT9_Confirm.setVisible(True)
            if self.MP_combo.currentIndex() == 3:
                labels = np.copy(labels35)
                df_concat = pd.concat([df[ageTimeBasedFeatures], pd.Series(labels, name='labels', index=df.index)], axis=1)
                cluster_ids_set = list(map(int, sorted(set(labels))))
                currentFeatures = ageTimeBasedFeatures.copy()
                ChangeCombo = False
                if self.P2_combo.count() > 5:
                    self.P2_combo.removeItem(5)
                self.C5_checkbox.setVisible(False)
                self.C5_checkbox.setChecked(False)
                self.P3_combo.clear()
                self.P3_combo.addItems(["--choose--"] + currentFeatures)
                self.P3_combo.view().setMinimumHeight(100)
                self.P4_combo.clear()
                self.P4_combo.addItems(["--choose--"] + currentFeatures)
                self.P4_combo.view().setMinimumHeight(100)
                ChangeCombo = True
                if lastGraphPage2 != 0:
                    method_name2 = f"plot_2_{lastGraphPage2}"
                    method2 = getattr(self, method_name2, None)
                    method2()
                if lastGraphPage3 != 0:
                    method_name3 = f"plot_3_{lastGraphPage3}"
                    method3 = getattr(self, method_name3, None)
                    method3()
                if lastGraphPage4 != 0:
                    method_name4 = f"plot_4_{lastGraphPage4}"
                    method4 = getattr(self, method_name4, None)
                    method4()
                self.resetAfterNewCluster()
                self.labelInfoCluster2.setText(
                    "Cluster 0: low lifetimedays, short term customers \n"
                    "Cluster 1: low age, young customers\n"
                    "Cluster 2: low first order, high lifetime -> early and loyal customers\n"
                    "Cluster 3: high age, old customers\n"
                    "Cluster 4: low first-/last order and lifetime -> early and short term customers")
                self.labelInfoCluster2.adjustSize()
                self.labelInfoCluster3.setText(
                    "Cluster 0: low lifetimedays, short term customers \n"
                    "Cluster 1: low age, young customers\n"
                    "Cluster 2: low first order, high lifetime -> early and loyal customers\n"
                    "Cluster 3: high age, old customers\n"
                    "Cluster 4: low first-/last order and lifetime -> early and short term customers")
                self.labelInfoCluster3.adjustSize()
                self.labelInfoCluster4.setText(
                    "Cluster 0: low lifetimedays, short term customers \n"
                    "Cluster 1: low age, young customers\n"
                    "Cluster 2: low first order, high lifetime -> early and loyal customers\n"
                    "Cluster 3: high age, old customers\n"
                    "Cluster 4: low first-/last order and lifetime -> early and short term customers")
                self.labelInfoCluster4.adjustSize()
                self.calc_info_label.setText("INFO: Other values will be calculated based on these:\n\n"
                                             "lifetime_days = last_order - first_order")

                self.set_relative_geometry_2(self.TT4_label, 500, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT4_input, 500, 400, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT4_Confirm, 500, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT4_err, 500, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT4_warn, 500, 500, 380, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT5_label, 750, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT5_input, 750, 400, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT5_Confirm, 750, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT5_err, 750, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT5_warn, 750, 500, 380, 40, base_width, base_height,
                                             current_width, current_height)

                self.set_relative_geometry_2(self.TT8_label, 250, 350, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT8_combo, 250, 400, 140, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT8_Confirm, 250, 450, 130, 30, base_width, base_height,
                                             current_width, current_height)
                self.set_relative_geometry_2(self.TT8_err, 250, 500, 430, 30, base_width, base_height,
                                             current_width, current_height)

                self.TT1_input.setVisible(False)
                self.TT1_label.setVisible(False)
                self.TT1_Confirm.setVisible(False)
                self.TT2_input.setVisible(False)
                self.TT2_label.setVisible(False)
                self.TT2_Confirm.setVisible(False)
                self.TT3_input.setVisible(False)
                self.TT3_label.setVisible(False)
                self.TT3_Confirm.setVisible(False)
                self.TT4_input.setVisible(True)
                self.TT4_label.setVisible(True)
                self.TT4_Confirm.setVisible(True)
                self.TT5_input.setVisible(True)
                self.TT5_label.setVisible(True)
                self.TT5_Confirm.setVisible(True)
                self.TT6_slider_label.setVisible(False)
                self.TT6_input_slider.setVisible(False)
                self.TT6_label.setVisible(False)
                self.TT6_Confirm.setVisible(False)
                self.TT7_slider_label.setVisible(False)
                self.TT7_input_slider.setVisible(False)
                self.TT7_label.setVisible(False)
                self.TT7_Confirm.setVisible(False)
                self.TT8_combo.setVisible(True)
                self.TT8_label.setVisible(True)
                self.TT8_Confirm.setVisible(True)
                self.TT9_input.setVisible(False)
                self.TT9_label.setVisible(False)
                self.TT9_Confirm.setVisible(False)


        if combo_name == "P2_combo":   #if the change comes from combobox on page 2
            if (lastGraph2 == 1):       #if last graph called was graph 1
                self.plot_2_1()     # call plot 1 again
            if (lastGraph2 == 2):
                self.plot_2_2()
            if (lastGraph2 == 3):
                self.plot_2_3()
        if combo_name == "P3_combo":    #if the change comes from combobox on page 3
            if (ChangeCombo == True):
                self.apply_stylesheet(self.P3_combo, border_color="white")
                if (lastGraph3 == 1):
                    self.plot_3_3()
        if combo_name == "P4_combo":    #if the change comes from combobox on page 4
            if (ChangeCombo == True):
                self.apply_stylesheet(self.P4_combo, border_color="white")
                if (lastGraph4 == 1):
                    self.plot_4_6()




    def on_text_change_input(self, label_name,input_name):  # method for when there is a change in inputfield, to reset its border and unconfirm it)
        global confirmed_status
        input = getattr(self, input_name)
        self.apply_stylesheet(input, border_color="white")
        confirmed_status[input_name] = False

    def on_checkbox_changed(self):   # method for when there is a change in checkbox, to call the last called plot again but with new checkbox values
        global lastGraphCheck3
        checked_count = 0
        if lastGraphCheck3 == 2:      # if last called graph was graph 2
            if self.C0_checkbox.isChecked():     # count which and how many checkboxes are checked
                checked_count += 1
            if self.C1_checkbox.isChecked():
                checked_count += 1
            if self.C2_checkbox.isChecked():
                checked_count += 1
            if self.C3_checkbox.isChecked():
                checked_count += 1
            if self.C4_checkbox.isChecked():
                checked_count += 1
            if self.C5_checkbox.isChecked():
                checked_count += 1

            if (checked_count != 2):     # if more or less then 2 are checked (this graph works with exactly 2 only)
                return
            else:
                self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")   # else call the plot and assure that border has normal color
                self.plot_3_2()

        elif lastGraphCheck3 == 1:    # if last called graph was 1
            if self.C0_checkbox.isChecked():
                checked_count += 1
            if self.C1_checkbox.isChecked():
                checked_count += 1
            if self.C2_checkbox.isChecked():
                checked_count += 1
            if self.C3_checkbox.isChecked():
                checked_count += 1
            if self.C4_checkbox.isChecked():
                checked_count += 1
            if self.C5_checkbox.isChecked():
                checked_count += 1

            if (checked_count == 0):  # if no box checked, do nothing
                return
            else:
                self.apply_stylesheet(self.showGraph3_1, border_color="lightgrey")   # else call the plot and reset border
                self.plot_3_1()

        elif lastGraphCheck3 == 3:
            if self.C0_checkbox.isChecked():
                checked_count += 1
            if self.C1_checkbox.isChecked():
                checked_count += 1
            if self.C2_checkbox.isChecked():
                checked_count += 1
            if self.C3_checkbox.isChecked():
                checked_count += 1
            if self.C4_checkbox.isChecked():
                checked_count += 1
            if self.C5_checkbox.isChecked():
                checked_count += 1

            if (checked_count == 0):
                return
            else:
                self.apply_stylesheet(self.showGraph3_3, border_color="lightgrey")
                self.plot_3_3()

        elif lastGraphCheck3 == 4:
            if self.C0_checkbox.isChecked():
                checked_count += 1
            if self.C1_checkbox.isChecked():
                checked_count += 1
            if self.C2_checkbox.isChecked():
                checked_count += 1
            if self.C3_checkbox.isChecked():
                checked_count += 1
            if self.C4_checkbox.isChecked():
                checked_count += 1
            if self.C5_checkbox.isChecked():
                checked_count += 1

            if (checked_count == 0):
                return
            else:
                self.apply_stylesheet(self.showGraph3_4, border_color="lightgrey")
                self.plot_3_4()

    def resetAfterNewCluster(self):   # resetting the page 5 after changing cluster in combobox
        confirmed_status['TT1_input'] = False
        confirmed_status['TT2_input'] = False
        confirmed_status['TT3_input'] = False
        confirmed_status['TT4_input'] = False
        confirmed_status['TT5_input'] = False
        confirmed_status['TT6_slider_label'] = False
        confirmed_status['TT7_slider_label'] = False
        confirmed_status['TT8_combo'] = False
        confirmed_status['TT9_input'] = False
        self.UnsualWarn.setVisible(False)
        self.pred_label.setVisible(False)
        self.FirstLast_err.setVisible(False)
        self.NotConfirmed_err.setVisible(False)
        self.cluster_calc_err.setVisible(False)
        self.apply_stylesheet(self.TT1_input, border_color='lightgrey')
        self.apply_stylesheet(self.TT2_input, border_color='lightgrey')
        self.apply_stylesheet(self.TT3_input, border_color='lightgrey')
        self.apply_stylesheet(self.TT4_input, border_color='lightgrey')
        self.apply_stylesheet(self.TT5_input, border_color='lightgrey')
        self.apply_stylesheet(self.TT6_slider_label, border_color='lightgrey')
        self.apply_stylesheet(self.TT7_slider_label, border_color='lightgrey')
        self.apply_stylesheet(self.TT8_combo, border_color='lightgrey')
        self.apply_stylesheet(self.TT9_input, border_color='lightgrey')
        self.TT1_err.setVisible(False)
        self.TT1_warn.setVisible(False)
        self.TT2_err.setVisible(False)
        self.TT2_warn.setVisible(False)
        self.TT3_err.setVisible(False)
        self.TT3_warn.setVisible(False)
        self.TT4_err.setVisible(False)
        self.TT4_warn.setVisible(False)
        self.TT5_err.setVisible(False)
        self.TT5_warn.setVisible(False)
        self.TT6_warn.setVisible(False)
        self.TT8_err.setVisible(False)
        self.TT9_err.setVisible(False)
        self.TT9_warn.setVisible(False)


    def update_label_slide_TT6(self, value):       # when there is a update in the label slider TT6
        global confirmed_status
        self.TT6_slider_label.setText(f"{value}")      # set the text of the slider label to the current value of slider
        self.TT6_slider_label.setStyleSheet("""
                            QLabel {
                                color: black;
                                background-color: white;
                                border: 2px solid gray;
                                border-radius: 5px;
                            }
                        """)
        confirmed_status["TT6_slider_label"] = False     # unconfirm the slider, because new value -> has to be confirmed again

    def update_label_slide_TT7(self, value):   # when there is a update in the label slider TT6
        global confirmed_status
        value = value / 100   # to show value as percent, because slider does not work with values lower than 1
        self.TT7_slider_label.setText(f"{value}")
        self.TT7_slider_label.setStyleSheet("""
                                    QLabel {
                                        color: black;
                                        background-color: white;
                                        border: 2px solid gray;
                                        border-radius: 5px;
                                    }
                                """)
        confirmed_status["TT7_slider_label"] = False





    def TT1_Confirm_Btn(self):     # when TT1 Confirm button is clicked
        global confirmed_status   # in order to change global values
        global warn_values
        global vendor_count
        self.TT1_warn.setVisible(False)   # set warn and error Labels invisible
        self.TT1_err.setVisible(False)
        warn_values['TT1_input'] = False   # currently no warning
        textTT1 = self.TT1_input.text()   # get text from input field
        if re.fullmatch(r"\d+", textTT1):  # check if input a positive number only
            if (int(textTT1) > 27):  # if higher than 90 --> unsual
                self.TT1_warn.setVisible(True)   # unsual value warning label visible
                warn_values['TT1_input'] = True  # warning in warn dictionary (in order to later show at calculate button that there are unsual values

            vendor_count = int(textTT1)    # declare value as vendor_count
            self.apply_stylesheet(self.TT1_input, border_color="lime")  # make border green because successfully confirmed
            confirmed_status['TT1_input'] = True   # confirm in confirm-dictionary
            self.TT1_err.setVisible(False)   # set error label invisible
        else:
            self.TT1_input.clear()  # if input not a positive number
            self.apply_stylesheet(self.TT1_input, border_color="red")    # border red
            self.TT1_err.setVisible(True) # error label visible

        # focus on next input field, for faster user experience (depending on current choosen clusterin approach from combobox)
        if self.MP_combo.currentIndex() == 0 or self.MP_combo.currentIndex() == 2:
            self.TT2_input.setFocus()
        elif self.MP_combo.currentIndex() == 1:
            self.TT7_input_slider.setFocus()

    def TT2_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global product_count
        self.TT2_warn.setVisible(False)
        self.TT2_err.setVisible(False)
        warn_values['TT2_input'] = False
        textTT2 = self.TT2_input.text()
        if re.fullmatch(r"[1-9]\d*", textTT2):  # check if input a positive number only, not 0
            if (int(textTT2) > 54):  # if higher than 54 --> unsual
                self.TT2_warn.setVisible(True)
                warn_values['TT2_input'] = True

            product_count = int(textTT2)
            self.apply_stylesheet(self.TT2_input, border_color="lime")
            confirmed_status['TT2_input'] = True
            self.TT2_err.setVisible(False)
        else:
            self.TT2_input.clear()  # if input not a positive number or 0
            self.apply_stylesheet(self.TT2_input, border_color="red")
            self.TT2_err.setVisible(True)
        # focus on next input field, for faster user experience (depending on current choosen clusterin approach from combobox)
        if self.MP_combo.currentIndex() == 0:
            self.TT3_input.setFocus()
        elif self.MP_combo.currentIndex() == 2:
            self.TT9_input.setFocus()

    def TT3_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global is_chain
        self.TT3_warn.setVisible(False)
        self.TT3_err.setVisible(False)
        warn_values['TT3_input'] = False
        textTT3 = self.TT3_input.text()
        if re.fullmatch(r"\d+", textTT3):  # check if input a positive number only
            if (int(textTT3) > 34):  # if higher than 34 --> unsual
                self.TT3_warn.setVisible(True)
                warn_values['TT3_input'] = True

            is_chain = int(textTT3)
            self.apply_stylesheet(self.TT3_input, border_color="lime")
            confirmed_status['TT3_input'] = True
            self.TT3_err.setVisible(False)
        else:
            self.TT3_input.clear()  # if input not a positive number
            self.apply_stylesheet(self.TT3_input, border_color="red")
            self.TT3_err.setVisible(True)
        self.TT9_input.setFocus()

    def TT4_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global first_order
        self.TT4_warn.setVisible(False)
        self.TT4_err.setVisible(False)
        warn_values['TT4_input'] = False
        textTT4 = self.TT4_input.text()
        if re.fullmatch(r"\d+", textTT4):  # check if input a positive number only
            if (int(textTT4) > 90):  # if higher than 90 --> unsual
                self.TT4_warn.setVisible(True)
                warn_values['TT4_input'] = True

            first_order = int(textTT4)
            self.apply_stylesheet(self.TT4_input, border_color="lime")
            confirmed_status['TT4_input'] = True
            self.TT4_err.setVisible(False)
        else:
            self.TT4_input.clear()  # if input not a positive number
            self.apply_stylesheet(self.TT4_input, border_color="red")
            self.TT4_err.setVisible(True)
        self.TT5_input.setFocus()

    def TT5_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global last_order
        self.TT5_warn.setVisible(False)
        self.TT5_err.setVisible(False)
        warn_values['TT5_input'] = False
        textTT5 = self.TT5_input.text()
        if re.fullmatch(r"\d+", textTT5):  # check if input a positive number only
            if (int(textTT5) > 90):  # if higher than 90 --> unsual
                self.TT5_warn.setVisible(True)
                warn_values['TT5_input'] = True

            last_order = int(textTT5)
            self.apply_stylesheet(self.TT5_input, border_color="lime")
            confirmed_status['TT5_input'] = True
            self.TT5_err.setVisible(False)
        else:
            self.TT5_input.clear()  # if input not a positive number
            self.apply_stylesheet(self.TT5_input, border_color="red")
            self.TT5_err.setVisible(True)
        # focus on next input field, for faster user experience (depending on current choosen clusterin approach from combobox)
        if self.MP_combo.currentIndex() == 0:
            self.TT6_input_slider.setFocus()
        elif self.MP_combo.currentIndex() == 3:
            self.calc_cluster_button.setFocus()

    def TT6_Confirm_Btn(self):
        global confirmed_status
        global total_expenses
        self.TT6_warn.setVisible(False)
        textTT6 = self.TT6_input_slider.value()   # get text from current slider value

        if (textTT6 > 328):    # if is higher than 328 --> unsual value
            self.TT6_warn.setVisible(True)
            warn_values['TT6_slider'] = True
        total_expenses = textTT6   # save in total_expenses variable
        confirmed_status['TT6_slider_label'] = True
        self.TT6_slider_label.setStyleSheet("""
                    QLabel {
                        color: black;
                        background-color: white;
                        border: 2px solid lime;
                        border-radius: 5px;
                    }
                """)
        # focus on next input field, for faster user experience (depending on current choosen clusterin approach from combobox)
        if self.MP_combo.currentIndex() == 0:
            self.TT7_input_slider.setFocus()
        elif self.MP_combo.currentIndex() == 2:
            self.calc_cluster_button.setFocus()

    def TT7_Confirm_Btn(self):
        global confirmed_status
        global culinary_variety
        textTT7 = self.TT7_input_slider.value()
        textTT7 = textTT7 / 100   # divide slider value by 100 to get percent, because slider does not work with values between 0 and 1

        culinary_variety = textTT7
        confirmed_status['TT7_slider_label'] = True
        self.TT7_slider_label.setStyleSheet("""
                    QLabel {
                        color: black;
                        background-color: white;
                        border: 2px solid lime;
                        border-radius: 5px;
                    }
                """)
        self.calc_cluster_button.setFocus()


    def TT8_Confirm_Btn(self):
        global confirmed_status
        global customer_age_group
        if (self.TT8_combo.currentText() == "---Choose---"):  # if ---Choose--- was chosen --> error label for choosing one
            self.TT8_err.setVisible(True)
            return
        elif (self.TT8_combo.currentText() == "15-17"):  # if this option was chosen --> set final value as 1
            textTT8 = 1
        elif (self.TT8_combo.currentText() == "18-22"):
            textTT8 = 2
        elif (self.TT8_combo.currentText() == "23-28"):
            textTT8 = 3
        elif (self.TT8_combo.currentText() == "29-35"):
            textTT8 = 4
        elif (self.TT8_combo.currentText() == "36-49"):
            textTT8 = 5
        elif (self.TT8_combo.currentText() == "50+"):
            textTT8 = 6

        customer_age_group = textTT8  # save in customer_age_group variable
        confirmed_status['TT8_combo'] = True  #confirm status True
        self.TT8_combo
        self.apply_stylesheet(self.TT8_combo, border_color='lime')
        self.TT4_input.setFocus()


    def TT9_Confirm_Btn(self):
        global confirmed_status
        global warn_values
        global OrdersPlaced
        self.TT9_warn.setVisible(False)
        self.TT9_err.setVisible(False)
        warn_values['TT9_input'] = False
        textTT9 = self.TT9_input.text()
        if re.fullmatch(r"[1-9]\d*", textTT9):  # check if input a positive number only and not 0
            if (int(textTT9) > 94):  # if higher than 94 --> unsual
                self.TT9_warn.setVisible(True)
                warn_values['TT9_input'] = True

            OrdersPlaced = int(textTT9)
            self.apply_stylesheet(self.TT9_input, border_color="lime")
            confirmed_status['TT9_input'] = True
            self.TT9_err.setVisible(False)
        else:
            self.TT9_input.clear()  # if input not a positive number or 0
            self.apply_stylesheet(self.TT9_input, border_color="red")
            self.TT9_err.setVisible(True)
        # focus on next input field, for faster user experience (depending on current choosen clusterin approach from combobox)
        if self.MP_combo.currentIndex() == 0:
            self.TT8_combo.setFocus()
        elif self.MP_combo.currentIndex() == 1:
            self.TT1_input.setFocus()
        elif self.MP_combo.currentIndex() == 2:
            self.TT6_input_slider.setFocus()


    def animate_loading(self):   # for the animation of loading label after clicking on 3d plot
        global is_growing  # a value that changes every 0.5 seconds
        scale_factor = min(current_width / base_width, current_height / base_height)
        new_fontsize_1 = int(28*scale_factor)  # different fontsizes calculated with scale factor
        new_fontsize_2 = int(20*scale_factor)
        new_padding = int(20*scale_factor)
        if is_growing:   # if currently is_growing == True
            self.load_label_3d.setStyleSheet(f"""
                                            QLabel {{
                                                border: 10px solid black;
                                                font-size: {new_fontsize_1}px;
                                                padding: {new_padding}px;
                                            }}
                                            """)
            is_growing = False   # set is_growing false
        else:   # if is_growing false
            self.load_label_3d.setStyleSheet(f"""
                                                QLabel {{
                                                    border: 5px solid black;
                                                    font-size: {new_fontsize_2}px;
                                                    padding: {new_padding}px;
                                                }}
                                                """)
            is_growing = True   # set is_growing true

    def calc_cluster(self, btn):  # method for when the add button to cluster is pressed
        global total_expenses
        global product_count
        global vendor_count
        global is_chain
        global first_order
        global last_order
        global culinary_variety
        global customer_age_group
        global OrdersPlaced
        self.UnsualWarn.setVisible(False)  # first set all warn/Error/Pred Label invisible
        self.pred_label.setVisible(False)
        self.FirstLast_err.setVisible(False)
        self.cluster_calc_err.setVisible(False)

        if self.MP_combo.currentIndex() == 1:   #if current approach choosen from combobox is preferencebased clustering
            confirmed_status['TT2_input'] = True   # confirm these values that will not be inputted (just necessary for scaling, values are not used)
            confirmed_status['TT4_input'] = True
            confirmed_status['TT5_input'] = True
            confirmed_status['TT6_slider_label'] = True
            confirmed_status['TT8_combo'] = True

            product_count = 5   # giving the mean values to these, again values not important just because scaler needs all features
            first_order = 28
            last_order = 63
            total_expenses = 35
            customer_age_group = 2

        if self.MP_combo.currentIndex() == 2:
            confirmed_status['TT3_input'] = True
            confirmed_status['TT4_input'] = True
            confirmed_status['TT5_input'] = True
            confirmed_status['TT7_slider_label'] = True
            confirmed_status['TT8_combo'] = True

            is_chain = 2
            first_order = 28
            last_order = 63
            culinary_variety = 0.14
            customer_age_group = 2

        if self.MP_combo.currentIndex() == 3:
            confirmed_status['TT1_input'] = True
            confirmed_status['TT2_input'] = True
            confirmed_status['TT3_input'] = True
            confirmed_status['TT6_slider_label'] = True
            confirmed_status['TT7_slider_label'] = True
            confirmed_status['TT9_input'] = True

            vendor_count = 3
            product_count = 5
            is_chain = 2
            total_expenses = 35
            culinary_variety = 0.14
            OrdersPlaced = 4

        if all(confirmed_status.values()): # if all fields are confirmed")
            if any(warn_values.values()):  # if there is any warning
                self.UnsualWarn.setVisible(True)  # set warnlabel visible
            self.NotConfirmed_err.setVisible(False)  # not confirmed error label invisible(because all values confirmed)

            if first_order > last_order:   # if first_order > last_order --> logic error
                self.FirstLast_err.setVisible(True)   # set error label visible
                self.apply_stylesheet(self.TT4_input, border_color="red")  # borders of first order and last order red
                self.apply_stylesheet(self.TT5_input, border_color="red")
                return  # skip rest of the method of button


            lifetime_days = last_order - first_order    # calculate these values needed for clustering, based on the input values
            avg_per_product = total_expenses / product_count
            avg_per_order = total_expenses / OrdersPlaced
            avg_order_size = product_count / OrdersPlaced
            chain_preference = is_chain / OrdersPlaced
            loyalty_to_venders = vendor_count / OrdersPlaced

            finalList = [vendor_count, product_count, is_chain, first_order, last_order, lifetime_days, total_expenses,
                         avg_per_product, avg_per_order, avg_order_size, culinary_variety, chain_preference, loyalty_to_venders,
                         customer_age_group]   # final list with all values for clustering (metric features)
            print("final_List:", finalList)
            dfDataAll = pd.DataFrame([finalList], columns=metric_features)   # make a dataframe with these values and metric_features as columns
            print("Head:", dfDataAll.head(1))

            dfDataAll = np.log1p(dfDataAll)   # use log function on the values
            print("headAfterLog:", dfDataAll.head(1))

            #scaling:
            dfDataAll[metric_features_scale] = scaler1.transform(dfDataAll[metric_features_scale]) # use scaler 1 for values who get scaled seperately

            group1_values = dfDataAll[group1].values.flatten().reshape(-1, 1)   # reshape values of Scale group 1
            transformed_values = scaler2.transform(group1_values)  # use sacler2 for group 1
            dfDataAll[group1] = transformed_values.reshape(-1, len(group1))  # reshape back

            group2_values = dfDataAll[group2].values.flatten().reshape(-1, 1)   # same for group 2
            transformed_values2 = scaler3.transform(group2_values)
            dfDataAll[group2] = transformed_values2.reshape(-1, len(group2))
            print("headAfterScaling:", dfDataAll.head(1))
            print(dfDataAll.columns)


            if btn == "calc1":   # if the left caluclate button was pressed (for quick prediction)
                # by calculating ward distance to cluster centroids the new point gets added to one of the clusters
                new_point = dfDataAll[currentFeatures].to_numpy()
                ward_distances = []

                if new_point.ndim > 1:  # if dimension too big
                    new_point = new_point.squeeze()
                for cluster in np.unique(labels):  # for each cluster
                    cluster_points = df[currentFeatures][labels == cluster]
                    n_C = len(cluster_points)

                    cluster_center = np.mean(cluster_points, axis=0) # calculate cluster center for each cluster (centroid)
                    distance = (n_C / (n_C + 1)) * (euclidean(new_point, cluster_center) ** 2)  # calculate distance between new point and centroid
                    ward_distances.append(distance) # add this distance to ward_distances list
                assigned_cluster = np.argmin(ward_distances)    # get the one with lowest distance as assigned cluster
                self.additional_code_after_clustering(assigned_cluster)

            if btn == "calc2":  # if the right button was pressed, for recalculating the clusters and detailled prediction
                global assignedCluster2
                new_point = dfDataAll[currentFeatures].to_numpy()

                df_copy = df[currentFeatures].copy()
                new_row = pd.DataFrame(new_point, columns=currentFeatures)
                df_copy = pd.concat([df_copy, new_row], ignore_index=True)  #df_copy is dataframe with new entry as last row

                if self.MP_combo.currentIndex() == 0:   # if the clustering approach chosen from combobox is FinalClustering (different process)
                    self.start_clustering(df_copy, 6, 0)
                else:  # if it is one of perspective clusterings (5 = number of clusters, 1 = information that it is one of perspectives
                    self.start_clustering(df_copy, 5, 1)

        else:  # if not all Fields confirmedd
            self.NotConfirmed_err.setVisible(True)  # make error label for not Confirmed visible
            self.pred_label.setVisible(False)  # final pred label invisible
            for key, value in confirmed_status.items():  # make the borders of all not confirmed fields darkblue
                if not value:
                    attribute = getattr(self, key, None)
                    if attribute:
                        self.apply_stylesheet(attribute, border_color="darkblue;", border_px="3px",
                                              border_status="solid")
            return


    def start_clustering(self, df, n_clusters, group):   # method to start the thread to clustering, group = 0 or 1 depending on choice of clus. approach
        self.load_label_5.setVisible(True)  # load label visible

        self.worker = ClusteringThread(df, n_clusters, group)  # create class instance
        self.worker.progress.connect(self.update_label_thread) # connect the signals from thread to methods
        self.worker.clustering_finished.connect(self.on_clustering_finished)
        self.worker.start()

    def update_label_thread(self, text):  # to update the text from loading label
        self.load_label_5.setText(text)

    def on_clustering_finished(self):  # when clustering thread finished
        global assignedCluster2
        print("assigAquiOnClusFInsiheed:", assignedCluster2)  # the assigned cluster (if it is 17 this is the default value so there was a mistake)

        self.load_label_5.setVisible(False) # loading label invisible
        assigned_c = assignedCluster2

        self.additional_code_after_clustering(assigned_c)  # call method for rest of code after clustering finished with the assgined cluster as argument

    def additional_code_after_clustering(self, assigned_cluster):
        print("assignedClusterHierAddCodeAFterClus", assigned_cluster)
        if self.MP_combo.currentIndex() == 0:  # if clustering approach choosen was final clustering
            if (assigned_cluster == 0):  # if it is cluster 0, set this in text of final pred label, and description of cluster
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"regular but more spending customers")
            if (assigned_cluster == 1):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"largest group of regular customers")
            if (assigned_cluster == 2):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"frequent orders but low loyality")
            if (assigned_cluster == 3):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"high frequency, small orders")
            if (assigned_cluster == 4):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"best spending customers")
            if (assigned_cluster == 5):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"least spending customers")
            self.pred_label.adjustSize()
            self.pred_label.setVisible(True)  # set the final result pred label visible

        if self.MP_combo.currentIndex() == 1:
            if (assigned_cluster == 0):  # if it is cluster 0, set this in text of final pred label, and description of cluster
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"regular with higher loyality and lower Chain-Orders")
            if (assigned_cluster == 1):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"high culinary variety and Chain-Orders")
            if (assigned_cluster == 2):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"low Chain-Orders, Chain-Preference and loyality")
            if (assigned_cluster == 3):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"regular with low loyality")
            if (assigned_cluster == 4):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"low Chain-Orders- and Preference but higher loyality")
            self.pred_label.adjustSize()
            self.pred_label.setVisible(True)  # set the final result pred label visible

        if self.MP_combo.currentIndex() == 2:
            if (
                    assigned_cluster == 0):  # if it is cluster 0, set this in text of final pred label, and description of cluster
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"regular orders and expenses, but high avgs p. Product/Order and Order Size")
            if (assigned_cluster == 1):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"low avg Ordersize, Product-/Vendorcount and total expenses")
            if (assigned_cluster == 2):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"high Product-/Vendorcount, low avgs -> more smaller orders")
            if (assigned_cluster == 3):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"very low avgs and total expenses, regular Product-/Vendorcount")
            if (assigned_cluster == 4):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"High Product-/Vendorcount, also high avgs -> frequent and big orders")
            self.pred_label.adjustSize()
            self.pred_label.setVisible(True)  # set the final result pred label visible

        if self.MP_combo.currentIndex() == 3:
            if (assigned_cluster == 0):  # if it is cluster 0, set this in text of final pred label, and description of cluster
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"low lifetime days, short term customers")
            if (assigned_cluster == 1):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"young customers")
            if (assigned_cluster == 2):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"low first order and high liftime -> early and loyal customers")
            if (assigned_cluster == 3):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"old customer")
            if (assigned_cluster == 4):
                self.pred_label.setText(f"New point was put in cluster {assigned_cluster}:\n"
                                        f"early and short term customer")
            self.pred_label.adjustSize()
            self.pred_label.setVisible(True)  # set the final result pred label visible

    def plot_2_1(self):   # method to plot plot_2_1 (from page 2)
        global lastGraph2
        global lastGraphPage2
        lastGraph2 = 1 # set the last called graph of page 2 as 1
        lastGraphPage2 = 1

        combo_value = int(self.P2_combo.currentText())   # get the value of combobox (for the cluster that is used)
        fig, ax = plt.subplots(figsize=(graph_size, graph_size))
        selected_cluster = combo_value
        cluster_data = df_concat[df_concat['labels'] == selected_cluster]

        sns.boxplot(data=cluster_data[currentFeatures], orient="h", ax=ax)  # create boxplot
        ax.set_title(f"Feature Distribution for Cluster {selected_cluster}")
        ax.set_xlabel("Feature Values")
        fig.tight_layout()

        canvas = FigureCanvas(fig)  # via canvas, show this plot in the label of page 2 to show grapgs
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label2.setPixmap(pixmap)
        self.graph_label2.adjustSize()

    def plot_2_2(self):   # method to plot plot_2_2 (from page 2)
        global lastGraph2
        global lastGraphPage2
        lastGraph2 = 2
        lastGraphPage2 = 2

        combo_value = int(self.P2_combo.currentText())
        fig, ax = plt.subplots(figsize=(graph_size, graph_size))
        selected_cluster = combo_value
        cluster_mean = df_concat[df_concat['labels'] == selected_cluster][currentFeatures].mean()

        sns.heatmap(cluster_mean.to_frame(), annot=True, cmap="coolwarm", cbar=False, ax=ax, fmt=".2f")
        ax.set_title(f"Feature Mean Values for Cluster {selected_cluster}")
        ax.set_xlabel("Features")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label2.setPixmap(pixmap)
        self.graph_label2.adjustSize()

    def plot_2_3(self):   # method to plot plot_2_3 (from page 2)
        global lastGraph2
        global lastGraphPage2
        lastGraph2 = 3
        lastGraphPage2 = 3

        combo_value = int(self.P2_combo.currentText())
        fig, ax = plt.subplots(figsize=(graph_size, graph_size))
        selected_cluster = combo_value
        cluster_data = df_concat[df_concat['labels'] == selected_cluster][currentFeatures]

        std_dev = cluster_data.std()
        std_dev.plot(kind='barh', ax=ax, color='darkblue', alpha=0.8)
        ax.set_title(f"Cluster Cohesion (Std Dev) for Cluster {selected_cluster}")
        ax.set_xlabel("Standard Deviation")
        ax.set_ylabel("Features")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label2.setPixmap(pixmap)
        self.graph_label2.adjustSize()

    def plot_3_1(self):  # method to plot plot_3_1 (from page 3)
        global lastGraph3
        global lastGraphPage3
        global lastGraphCheck3

        lastGraphCheck3 = 1   # last called graph of page 3 to 1 (necessary for calling last called plot after checkbox change)
        lastGraph3 = 0   # set last called graph of page 3 to 0 (only set to 1 for plot 3_3 (who needs value from combobox))
        lastGraphPage3 = 1

        checked_list = []  # to count who many and which checkboxes are checked
        checked_count = 0
        self.P3_err.setVisible(False)
        self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")   # reset border of graphButton 3_2

        if self.C0_checkbox.isChecked(): # count how many and which checkboxes are checked
            checked_count += 1
            checked_list.append(0)
        if self.C1_checkbox.isChecked():
            checked_count += 1
            checked_list.append(1)
        if self.C2_checkbox.isChecked():
            checked_count += 1
            checked_list.append(2)
        if self.C3_checkbox.isChecked():
            checked_count += 1
            checked_list.append(3)
        if self.C4_checkbox.isChecked():
            checked_count += 1
            checked_list.append(4)
        if self.C5_checkbox.isChecked():
            checked_count += 1
            checked_list.append(5)

        if (checked_count == 0):  # if 0 choosen
            self.P3_err.setText("Choose at least one cluster")  # error label visible
            self.apply_stylesheet(self.showGraph3_1, border_color="red")  # border of this button red
            self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey") # border of other buttons reset
            self.apply_stylesheet(self.showGraph3_3, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_4, border_color="lightgrey")
            self.P3_err.setVisible(True)
        else:
            fig, ax = plt.subplots(figsize=(graph_size, graph_size), subplot_kw={'polar': True})    # if > 1, call the plot

            cluster_ids = checked_list
            cluster_means = df_concat.groupby('labels').mean().loc[cluster_ids]

            categories = currentFeatures
            N = len(categories)

            values = cluster_means.values.T
            values = np.concatenate((values, values[:1, :]), axis=0)

            angles = [n / float(N) * 2 * pi for n in range(N)]
            angles += angles[:1]

            angles = angles[::-1]

            plt.xticks(angles[:-1], categories)

            for idx, cluster_id in enumerate(cluster_ids):
                ax.plot(angles, values[:, idx], label=f'Cluster {cluster_id}')
                ax.fill(angles, values[:, idx], alpha=0.25)

            ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
            ax.set_title("Radar Chart: Comparing Clusters")
            fig.tight_layout()

            canvas = FigureCanvas(fig)
            canvas.draw()
            img = canvas.buffer_rgba()
            qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
            pixmap = QtGui.QPixmap.fromImage(qim)
            self.graph_label3.setPixmap(pixmap)
            self.graph_label3.adjustSize()

    def plot_3_2(self):      # method to plot plot_3_2 (from page 3)
        global lastGraph3
        global lastGraphCheck3
        global lastGraphPage3
        lastGraphCheck3 = 2
        lastGraph3 = 0 # only 1 when graph 3_3 (because needs combobox value)
        lastGraphPage3 = 2

        checked_list = []
        checked_count = 0
        self.P3_err.setVisible(False)

        if self.C0_checkbox.isChecked():
            checked_count += 1
            checked_list.append(0)
        if self.C1_checkbox.isChecked():
            checked_count += 1
            checked_list.append(1)
        if self.C2_checkbox.isChecked():
            checked_count += 1
            checked_list.append(2)
        if self.C3_checkbox.isChecked():
            checked_count += 1
            checked_list.append(3)
        if self.C4_checkbox.isChecked():
            checked_count += 1
            checked_list.append(4)
        if self.C5_checkbox.isChecked():
            checked_count += 1
            checked_list.append(5)

        if(checked_count != 2):
            self.P3_err.setText("Choose exactly two clusters")
            self.apply_stylesheet(self.showGraph3_2, border_color="red")
            self.apply_stylesheet(self.showGraph3_1, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_3, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_4, border_color="lightgrey")
            self.P3_err.setVisible(True)
        else:
            fig, ax = plt.subplots(figsize=(graph_size, graph_size))
            cluster_ids = checked_list
            cluster_means = df_concat.groupby('labels').mean()

            diff = cluster_means.loc[cluster_ids[1]] - cluster_means.loc[cluster_ids[0]]
            diff.plot(kind='bar', ax=ax, color='darkblue', alpha=0.7)

            ax.axhline(0, color='black', linewidth=1, linestyle='--')
            ax.set_title(f"Feature Differences Between Clusters {cluster_ids[0]} and {cluster_ids[1]}")
            ax.set_ylabel("Difference in Feature Means")
            fig.tight_layout()

            canvas = FigureCanvas(fig)
            canvas.draw()
            img = canvas.buffer_rgba()
            qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
            pixmap = QtGui.QPixmap.fromImage(qim)
            self.graph_label3.setPixmap(pixmap)
            self.graph_label3.adjustSize()

    def plot_3_3(self):  # method to plot plot_3_3 (from page 3)
        global lastGraph3
        global lastGraphCheck3
        global lastGraphPage3
        lastGraphCheck3 = 3
        lastGraph3 = 1  # set as 1 because this graph needs value from combobox
        lastGraphPage3 = 3

        checked_list = []
        checked_count = 0
        self.P3_err.setVisible(False)
        self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")

        if self.C0_checkbox.isChecked():
            checked_count += 1
            checked_list.append(0)
        if self.C1_checkbox.isChecked():
            checked_count += 1
            checked_list.append(1)
        if self.C2_checkbox.isChecked():
            checked_count += 1
            checked_list.append(2)
        if self.C3_checkbox.isChecked():
            checked_count += 1
            checked_list.append(3)
        if self.C4_checkbox.isChecked():
            checked_count += 1
            checked_list.append(4)
        if self.C5_checkbox.isChecked():
            checked_count += 1
            checked_list.append(5)

        if (checked_count == 0):
            self.P3_err.setText("Choose at least one cluster")
            self.apply_stylesheet(self.showGraph3_3, border_color="red")
            self.apply_stylesheet(self.showGraph3_1, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_4, border_color="lightgrey")
            self.P3_err.setVisible(True)
        else:
            current_index = self.P3_combo.currentIndex()    # get value from combobox
            if current_index != 0:
                current_index -= 1
                fig, ax = plt.subplots(figsize=(graph_size, graph_size))
                selected_feature = currentFeatures[current_index]
                cluster_ids = checked_list

                for cluster_id in cluster_ids:
                    sns.kdeplot(
                        df_concat[df_concat['labels'] == cluster_id][selected_feature],
                        ax=ax, label=f"Cluster {cluster_id}", fill=True, alpha=0.5
                    )

                ax.set_title(f"Distribution Overlap for Feature: {selected_feature}")
                ax.set_xlabel(selected_feature)
                ax.legend()
                fig.tight_layout()

                canvas = FigureCanvas(fig)
                canvas.draw()
                img = canvas.buffer_rgba()
                qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
                pixmap = QtGui.QPixmap.fromImage(qim)
                self.graph_label3.setPixmap(pixmap)
                self.graph_label3.adjustSize()
            else:
                self.apply_stylesheet(self.P3_combo, border_color="red")
                return

    def plot_3_4(self):  # method to plot plot_3_4 (from page 3)
        global lastGraph3
        global lastGraphCheck3
        global lastGraphPage3
        lastGraphCheck3 = 4
        lastGraph3 = 0   # only 1 for 3_3 because needs combobox value
        lastGraphPage3 = 4

        checked_list = []
        checked_count = 0
        self.P3_err.setVisible(False)
        self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")

        if self.C0_checkbox.isChecked():
            checked_count += 1
            checked_list.append(0)
        if self.C1_checkbox.isChecked():
            checked_count += 1
            checked_list.append(1)
        if self.C2_checkbox.isChecked():
            checked_count += 1
            checked_list.append(2)
        if self.C3_checkbox.isChecked():
            checked_count += 1
            checked_list.append(3)
        if self.C4_checkbox.isChecked():
            checked_count += 1
            checked_list.append(4)
        if self.C5_checkbox.isChecked():
            checked_count += 1
            checked_list.append(5)

        if(checked_count == 0):
            self.P3_err.setText("Choose at least one cluster")
            self.P3_err.setVisible(True)
            self.apply_stylesheet(self.showGraph3_4, border_color="red")
            self.apply_stylesheet(self.showGraph3_1, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_2, border_color="lightgrey")
            self.apply_stylesheet(self.showGraph3_3, border_color="lightgrey")
        else:
            fig, ax = plt.subplots(figsize=(graph_size, graph_size))
            cluster_ids = checked_list
            cluster_centers = df_concat.groupby('labels')[currentFeatures].mean().loc[cluster_ids]

            distances = cdist(
                df_concat[df_concat['labels'].isin(cluster_ids)][currentFeatures],
                cluster_centers, metric='euclidean'
            )

            distances_df = pd.DataFrame(distances, columns=[f"Cluster {id}" for id in cluster_ids])
            sns.boxplot(data=distances_df, ax=ax)

            ax.set_title("Distance Distribution Between Points and Cluster Centers")
            ax.set_ylabel("Euclidean Distance")
            fig.tight_layout()

            canvas = FigureCanvas(fig)
            canvas.draw()
            img = canvas.buffer_rgba()
            qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
            pixmap = QtGui.QPixmap.fromImage(qim)
            self.graph_label3.setPixmap(pixmap)
            self.graph_label3.adjustSize()


    def plot_4_1(self):   # method to plot plot_4_1 (from page 4)
        global lastGraph4
        global lastGraphPage4
        lastGraph4 = 0  # only 1 for 4_6, because needs value from combobox
        lastGraphPage4 = 1

        self.load_label_3d.setVisible(False)  # loading label (for 3dplot) invisible
        self.graph_label4.setVisible(True)   # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()   # hide label from 3d plot
        fig, ax = plt.subplots(figsize=(graph_size, graph_size))
        hc_profile = df_concat.groupby('labels').mean().T
        sns.heatmap(hc_profile,center=0, annot=True, cmap="BrBG", fmt=".2f",ax=ax)
        ax.set_xlabel("Cluster Labels")
        ax.set_title("Cluster Profiling:\nHierarchical Clustering")
        fig.tight_layout()
        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()

    def plot_4_2(self):  # method to plot plot_4_2 (from page 4)
        global lastGraph4
        global lastGraphPage4
        lastGraph4 = 0    # only 1 for 4_6, because needs value from combobox
        lastGraphPage4 = 2

        self.load_label_3d.setVisible(False)   # load label invisible
        self.graph_label4.setVisible(True)    # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()  # 3d plot label invisible
        fig, ax = plt.subplots(figsize=(graph_size,graph_size))
        sns.countplot(data=df_concat, x='labels')
        ax.set_title("Cluster Sizes")
        ax.set_xlabel("Cluster Labels")
        fig.tight_layout()
        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()


    def plot_4_3(self):   # method to plot plot_4_3 (from page 4)
        global lastGraph4
        global lastGraphPage4
        lastGraph4 = 0     # only 1 for 4_6, because needs value from combobox
        lastGraphPage4 = 3

        self.load_label_3d.setVisible(False)  # load label invisible
        self.graph_label4.setVisible(True)   # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()    # 3d plot label invisible
        pca = PCA(n_components=2)
        reduced_data = pca.fit_transform(df[currentFeatures])
        cluster_labels = labels
        plt.figure(figsize=(8, 6))
        fig, ax = plt.subplots(figsize=(graph_size, graph_size))
        for cluster in range(clusters.n_clusters):
            cluster_points = reduced_data[cluster_labels == cluster]
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {cluster}')
        ax.set_xlabel('PCA Component 1')
        ax.set_ylabel('PCA Component 2')
        ax.set_title('Cluster Visualization with PCA')
        ax.legend()
        fig.tight_layout()
        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()


    def plot_4_4(self):  # method to plot plot_4_4 (from page 4)
        global lastGraph4
        lastGraph4 = 0     # only 1 for 4_6, because needs value from combobox
        global lastGraphPage4
        lastGraphPage4 = 4

        self.load_label_3d.setVisible(False)  # load label invisible
        self.graph_label4.setVisible(True) # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()  # 3d plot label invisible
        importances = []

        X = df[currentFeatures]
        y = labels
        clf = DecisionTreeClassifier(random_state=42)
        clf.fit(X, y)

        importances = pd.Series(clf.feature_importances_, index=currentFeatures).sort_values(ascending=False)

        fig, ax = plt.subplots(figsize=(graph_size, graph_size))
        importances.plot(kind='bar', ax=ax, color='darkblue', alpha=0.7)
        ax.set_title("Feature Importance Across Clusters")
        ax.set_ylabel("Importance")
        ax.set_xlabel("Features")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()


    def plot_4_5(self):    # method to plot plot_4_5 (from page 4)
        global lastGraph4
        lastGraph4 = 0     # only 1 for 4_6, because needs value from combobox
        global lastGraphPage4
        lastGraphPage4 = 5

        self.load_label_3d.setVisible(False)  # load label invisible
        self.graph_label4.setVisible(True)    # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide() # 3d plot label invisible
        fig, ax = plt.subplots(figsize=(graph_size, graph_size), subplot_kw={'polar': True})

        cluster_ids = cluster_ids_set
        cluster_means = df_concat.groupby('labels').mean().loc[cluster_ids]

        categories = currentFeatures
        N = len(categories)

        values = cluster_means.values.T
        values = np.concatenate((values, values[:1, :]), axis=0)

        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]

        angles = angles[::-1]

        plt.xticks(angles[:-1], categories)

        for idx, cluster_id in enumerate(cluster_ids):
            ax.plot(angles, values[:, idx], label=f'Cluster {cluster_id}')
            ax.fill(angles, values[:, idx], alpha=0.25)

        ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        ax.set_title("Radar Chart: Comparing Clusters")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()

    def plot_4_6(self):      # method to plot plot_4_6 (from page 4)
        global lastGraph4
        lastGraph4 = 1     # only 1 for 4_6, because needs value from combobox
        global lastGraphPage4
        lastGraphPage4 = 6

        self.load_label_3d.setVisible(False)  # load label invisible
        self.graph_label4.setVisible(True)    # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()  # 3d plot label invisible
        current_index = self.P4_combo.currentIndex()
        if current_index != 0:
            current_index -= 1
            fig, ax = plt.subplots(figsize=(graph_size, graph_size))
            selected_feature = currentFeatures[current_index]
            cluster_ids = cluster_ids_set

            for cluster_id in cluster_ids:
                sns.kdeplot(
                    df_concat[df_concat['labels'] == cluster_id][selected_feature],
                    ax=ax, label=f"Cluster {cluster_id}", fill=True, alpha=0.5
                )

            ax.set_title(f"Distribution Overlap for Feature: {selected_feature}")
            ax.set_xlabel(selected_feature)
            ax.legend()
            fig.tight_layout()

            canvas = FigureCanvas(fig)
            canvas.draw()
            img = canvas.buffer_rgba()
            qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
            pixmap = QtGui.QPixmap.fromImage(qim)
            self.graph_label4.setPixmap(pixmap)
            self.graph_label4.adjustSize()
        else:
            self.apply_stylesheet(self.P4_combo, border_color="red")
            return

    def plot_4_7(self):      # method to plot plot_4_7 (from page 4)
        global lastGraph4
        lastGraph4 = 0       # only 1 for 4_6, because needs value from combobox
        global lastGraphPage4
        lastGraphPage4 = 7

        self.load_label_3d.setVisible(False)   # load label invisible
        self.graph_label4.setVisible(True)  # graph label for page 4 visible (because 3d plot gets on same position, so when not 3d plot this label has to return
        self.web_view.hide()  # 3d plot label invisible
        fig, ax = plt.subplots(figsize=(graph_size, graph_size))
        cluster_ids = cluster_ids_set
        cluster_centers = df_concat.groupby('labels')[currentFeatures].mean().loc[cluster_ids]

        distances = cdist(
            df_concat[df_concat['labels'].isin(cluster_ids)][currentFeatures],
            cluster_centers, metric='euclidean'
        )

        distances_df = pd.DataFrame(distances, columns=[f"Cluster {id}" for id in cluster_ids])

        sns.boxplot(data=distances_df, ax=ax)

        ax.set_title("Distance Distribution Between Points and Cluster Centers")
        ax.set_ylabel("Euclidean Distance")
        fig.tight_layout()

        canvas = FigureCanvas(fig)
        canvas.draw()
        img = canvas.buffer_rgba()
        qim = QtGui.QImage(img, canvas.width(), canvas.height(), QtGui.QImage.Format_RGBA8888)
        pixmap = QtGui.QPixmap.fromImage(qim)
        self.graph_label4.setPixmap(pixmap)
        self.graph_label4.adjustSize()

    def on_load_finished(self):
        self.load_label_3d.setVisible(False)

    def plot_4_8(self):         # method to plot plot_4_8 (from page 4), 3d Plot
        global lastGraph4
        lastGraph4 = 0       # only 1 for 4_6, because needs value from combobox
        global lastGraphPage4
        lastGraphPage4 = 8

        self.load_label_3d.setVisible(False)   # load label invisible (to reset it)
        self.load_label_3d.setVisible(True) # load label visible (until the plot is fully loaded)

        pca = PCA(n_components=3)   # for plot
        principal_components = pca.fit_transform(df[currentFeatures])
        pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3'])
        pca_df['cluster'] = labels
        fig = px.scatter_3d(pca_df, x='PC1', y='PC2', z='PC3', title="PCA 3D Plot (PC1, PC2, PC3)",color='cluster',
                            labels={'PC1': 'PC1', 'PC2': 'PC2', 'PC3': 'PC3'})
        html_content = fig.to_html(full_html=False)   # create plot via html (in order to interact with 3d plot)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as tmp_file:   # save plot as temporary file
            tmp_file.write(html_content.encode('utf-8'))
            temp_html_path = tmp_file.name

        self.web_view.setGeometry(self.graph_label4.geometry())   # create web view label in place of graph_label4
        self.web_view.setFixedSize(graph_size*100, graph_size*100)
        self.web_view.loadFinished.connect(self.on_load_finished)
        self.web_view.load(QUrl.fromLocalFile(temp_html_path))
        self.web_view.show()
        self.graph_label4.setVisible(False)   # make graph label 4 invisible


    def setupUi(self, MainWindow):    # to create buttons, labels etc
        MainWindow.setObjectName("MainWindow")  # create main window

        MainWindow.resize(current_width, current_height)

        MainWindow.setStyleSheet("background-color: white;")

        self.stacked_widget = QtWidgets.QStackedWidget(MainWindow)
        MainWindow.setCentralWidget(self.stacked_widget)


        self.MP_combo = QtWidgets.QComboBox(MainWindow)
        self.MP_combo.addItem("Final Clustering")
        self.MP_combo.addItem("Preference Based Features")
        self.MP_combo.addItem("Purchase Based Features")
        self.MP_combo.addItem("Age/Time Based Features")
        self.MP_combo.setObjectName("MP_combo")
        self.MP_combo.setStyleSheet("""
                                QComboBox {
                                        background-color: white;
                                        color: black;
                                        border: 2px solid white;
                                        border-radius: 10px;
                                    }
                                QComboBox:focus {
                                    border: 2px solid blue;
                                }
                                QComboBox:editable:on {
                                    border: 2px solid blue;
                                }
                                QComboBox:!editable:on {
                                    border: 2px solid blue;
                                }
                                QComboBox QAbstractItemView {
                                            background-color: white;
                                            color: black;
                                            border: 2px solid white;
                                            selection-background-color: lightblue;
                                            selection-color: black;
                                }
                                            """)

        self.MP_combo.view().setMinimumWidth(160)
        self.set_relative_geometry_3(self.MP_combo, 820, 170, 200, 60, base_width, base_height, current_width,
                                     current_height, fontsize=22, paddingsize=8)
        self.MP_combo.currentIndexChanged.connect(lambda: self.on_combobox_change_2("MP_combo"))


        self.page1 = QtWidgets.QWidget()   # create pages (part of stacked_widget)
        self.setupPage1(self.page1)

        self.page2 = QtWidgets.QWidget()
        self.setupPage2(self.page2)

        self.page3 = QtWidgets.QWidget()
        self.setupPage3(self.page3)

        self.page4 = QtWidgets.QWidget()
        self.setupPage4(self.page4)

        self.page5 = QtWidgets.QWidget()
        self.setupPage5(self.page5)

        self.stacked_widget.addWidget(self.page1)   # add pages to stacked_widget
        self.stacked_widget.addWidget(self.page2)
        self.stacked_widget.addWidget(self.page3)
        self.stacked_widget.addWidget(self.page4)
        self.stacked_widget.addWidget(self.page5)

        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)


        #---------------------


    def setupPage1(self, page):   # method for page 1
        page.setObjectName("Page1")
        page.setStyleSheet("background-color: lightblue;")


        # Label page 1
        self.label1 = QtWidgets.QLabel(page)
        self.label1.setText("Main Page")
        self.label1.setAlignment(QtCore.Qt.AlignCenter)
        self.label1.setStyleSheet("""
                    QLabel {
                         background-color: lightgray;
                         color: black;
                         border: 2px solid white;
                         border-radius: 10px;
                     }
        """)
        self.set_relative_geometry_3(self.label1, 820, 40, 200, 80, base_width, base_height, current_width,
                                     current_height, paddingsize=30, fontsize=40)



        # Button to page 2
        self.to_page2_button = QtWidgets.QPushButton(page)
        self.to_page2_button.setText("Insights in one cluster")
        self.to_page2_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.to_page2_button, 550, 250, 300, 150, base_width, base_height, current_width,
                                     current_height, paddingsize=25, fontsize=24)
        self.to_page2_button.clicked.connect(self.go_to_page2)

        # Button to page 3
        self.to_page3_button = QtWidgets.QPushButton(page)
        self.to_page3_button.setText("Compare clusters")
        self.to_page3_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.to_page3_button, 1000, 250, 300, 150, base_width, base_height, current_width,
                                     current_height, paddingsize=25, fontsize=24)
        self.to_page3_button.clicked.connect(self.go_to_page3)

        # Button to page 4
        self.to_page4_button = QtWidgets.QPushButton(page)
        self.to_page4_button.setText("Insights into all clusters")
        self.to_page4_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.to_page4_button, 550, 500, 300, 150, base_width, base_height, current_width,
                                     current_height, paddingsize=25, fontsize=24)
        self.to_page4_button.clicked.connect(self.go_to_page4)

        # Button to page 5
        self.to_page5_button = QtWidgets.QPushButton(page)
        self.to_page5_button.setText("Connect new entry\nto cluster")
        self.to_page5_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.to_page5_button, 1000, 500, 300, 150, base_width, base_height, current_width,
                                     current_height, paddingsize=25, fontsize=24)
        self.to_page5_button.clicked.connect(self.go_to_page5)




    def go_to_page1(self):
        self.stacked_widget.setCurrentWidget(self.page1)
        self.set_relative_geometry_3(self.MP_combo, 820, 170, 200, 60, base_width, base_height, current_width,
                                     current_height, fontsize=22, paddingsize=8)

    def setupPage2(self, page):
        page.setObjectName("Page2")
        page.setStyleSheet("background-color: lightblue;")



        # Label Info Cluster page 2
        self.labelInfoCluster2 = QtWidgets.QLabel(page)
        self.labelInfoCluster2.setText(
            "Cluster 0: regular but more spending customers\n"
            "Cluster 1: largest group of regular customers\n"
            "Cluster 2: frequent orders but low loyality\n"
            "Cluster 3: high frequency, small orders\n"
            "Cluster 4: best spending customers\n"
            "Cluster 5: least spending customers")
        self.labelInfoCluster2.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         border-radius: 10px;
                                     }
                        """)
        self.set_relative_geometry_3(self.labelInfoCluster2, 50, 650, 500, 130, base_width, base_height, current_width,
                                     current_height, paddingsize=5, fontsize=22)

        # Label for page 2
        self.label2 = QtWidgets.QLabel(page)
        self.label2.setText("Insights in one cluster")
        self.label2.setStyleSheet("""
                            QLabel {
                                 background-color: lightgray;
                                 color: black;
                                 border: 2px solid white;
                                 border-radius: 10px;
                             }
                """)
        self.set_relative_geometry_3(self.label2, 450, 50, 300, 30, base_width, base_height, current_width,
                                     current_height, paddingsize=5, fontsize=28)


        self.to_page1_button = QtWidgets.QPushButton(page)
        self.to_page1_button.setText("back")
        self.to_page1_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.to_page1_button, 50, 150, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.to_page1_button.clicked.connect(self.go_to_page1)

        # Label for combo
        self.combo_label2 = QtWidgets.QLabel(page)
        self.combo_label2.setText("Choose Cluster")
        self.combo_label2.setStyleSheet("border: 0px solid black;")
        self.set_relative_geometry_3(self.combo_label2, 750, 245, 100, 100, base_width, base_height, current_width,
                                     current_height, fontsize=16)

        # ----- 2 Combo Box -----
        self.P2_combo = QComboBox(page)
        self.P2_combo.addItem("0")
        self.P2_combo.addItem("1")
        self.P2_combo.addItem("2")
        self.P2_combo.addItem("3")
        self.P2_combo.addItem("4")
        self.P2_combo.addItem("5")
        self.P2_combo.setObjectName("P2_combo")
        self.P2_combo.setStyleSheet("""
                                        QComboBox {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                        QComboBox:focus {
                                            border: 2px solid blue;
                                        }
                                        QComboBox:editable:on {
                                            border: 2px solid blue;
                                        }
                                        QComboBox:!editable:on {
                                            border: 2px solid blue;
                                        }
                                        QComboBox QAbstractItemView {
                                                    background-color: white;
                                                    color: black;
                                                    border: 2px solid white;
                                                    selection-background-color: lightblue;
                                                    selection-color: black;
                                        }
                                                    """)

        self.set_relative_geometry_3(self.P2_combo, 750, 270, 180, 40, base_width, base_height, current_width,
                                     current_height)
        self.P2_combo.setMinimumWidth(100)
        self.P2_combo.currentIndexChanged.connect(lambda: self.on_combobox_change_2("P2_combo"))

        # Button Show Graph 2_1
        self.showGraph2_1 = QtWidgets.QPushButton(page)
        self.showGraph2_1.setText("Cluster boxplot")
        self.showGraph2_1.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph2_1, 750, 350, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph2_1.clicked.connect(self.plot_2_1)

        # Button Show Graph 2_2
        self.showGraph2_2 = QtWidgets.QPushButton(page)
        self.showGraph2_2.setText("Cluster heatmap")
        self.showGraph2_2.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph2_2, 750, 450, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph2_2.clicked.connect(self.plot_2_2)

        # Button Show Graph 2_3
        self.showGraph2_3 = QtWidgets.QPushButton(page)
        self.showGraph2_3.setText("Cluster Coherence")
        self.showGraph2_3.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph2_3, 750, 550, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph2_3.clicked.connect(self.plot_2_3)

        # Label for graph 2
        self.graph_label2 = QtWidgets.QLabel(page)
        self.set_relative_geometry_3(self.graph_label2, 1080, 100, 800, 800, base_width, base_height, current_width,
                                     current_height)
        self.graph_label2.setStyleSheet("border: 0px solid black;")

    def go_to_page2(self):
        self.stacked_widget.setCurrentWidget(self.page2)
        self.set_relative_geometry_3(self.MP_combo, 500, 160, 230, 60, base_width, base_height, current_width,
                                     current_height, fontsize=22, paddingsize=8)


    def setupPage3(self, page):
        page.setObjectName("Page3")
        page.setStyleSheet("background-color: lightblue;")


        self.labelInfoCluster3 = QtWidgets.QLabel(page)
        self.labelInfoCluster3.setText(
            "Cluster 0: regular but more spending customers\n"
            "Cluster 1: largest group of regular customers\n"
            "Cluster 2: frequent orders but low loyality\n"
            "Cluster 3: high frequency, small orders\n"
            "Cluster 4: best spending customers\n"
            "Cluster 5: least spending customers")
        self.labelInfoCluster3.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         border-radius: 10px;
                                     }
                        """)
        self.set_relative_geometry_3(self.labelInfoCluster3, 50, 650, 500, 130, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        self.label3 = QtWidgets.QLabel(page)
        self.label3.setText("Compare clusters")
        self.label3.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         border-radius: 10px;
                                     }
                        """)
        self.set_relative_geometry_3(self.label3, 450, 50, 300, 30, base_width, base_height, current_width,
                                     current_height, fontsize=28)


        self.to_page1_button = QtWidgets.QPushButton(page)
        self.to_page1_button.setText("back")
        self.to_page1_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.to_page1_button, 50, 150, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.to_page1_button.clicked.connect(self.go_to_page1)

        # ---- The Page3 Error Label
        self.P3_err = QtWidgets.QLabel(page)
        self.P3_err.setText("----------------------------------")
        self.P3_err.setVisible(False)
        self.P3_err.setStyleSheet("""
                                    QLabel {
                                        background-color: lightgray;
                                        color: black;
                                        border: 2px solid red;
                                        border-radius: 10px;
                                    }
                                        """)
        self.set_relative_geometry_3(self.P3_err, 590, 310, 230, 30, base_width, base_height, current_width,
                                     current_height)



        # ----- 3 Combo Box -----
        self.P3_combo = QComboBox(page)
        self.P3_combo.addItems(["--choose--"] + currentFeatures)
        self.P3_combo.setObjectName("P3_combo")
        self.P3_combo.setStyleSheet("""
                                                QComboBox {
                                                        background-color: white;
                                                        color: black;
                                                        border: 2px solid white;
                                                        border-radius: 10px;
                                                    }
                                                QComboBox:focus {
                                                    border: 2px solid blue;
                                                }
                                                QComboBox:editable:on {
                                                    border: 2px solid blue;
                                                }
                                                QComboBox:!editable:on {
                                                    border: 2px solid blue;
                                                }
                                                QComboBox QAbstractItemView {
                                                            background-color: white;
                                                            color: black;
                                                            border: 2px solid white;
                                                            selection-background-color: lightblue;
                                                            selection-color: black;
                                                }
                                                            """)

        self.P3_combo.view().setMinimumWidth(200)
        self.P3_combo.view().setMinimumHeight(300)
        self.set_relative_geometry_3(self.P3_combo, 450, 510, 180, 40, base_width, base_height, current_width,
                                     current_height)
        self.P3_combo.currentIndexChanged.connect(lambda: self.on_combobox_change_2("P3_combo"))

        # ----- C0 Checkbox -----
        self.C0_checkbox = QtWidgets.QCheckBox(page)
        self.C0_checkbox.setText("C0")
        self.C0_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C0_checkbox.setStyleSheet("""
                            QCheckBox {
                                color: black;
                                border-radius: 10px;
                            }
                            QCheckBox::indicator {
                                width: 30px;
                                height: 30px;
                            }
                        """)
        self.set_relative_geometry_3(self.C0_checkbox, 400, 250, 74, 50, base_width, base_height, current_width,
                                     current_height, fontsize=16, paddingsize=0)

        # ----- C1 Checkbox -----
        self.C1_checkbox = QtWidgets.QCheckBox(page)
        self.C1_checkbox.setText("C1")
        self.C1_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C1_checkbox.setStyleSheet("""
                                    QCheckBox {
                                        color: black;
                                        border-radius: 10px;
                                    }
                                    QCheckBox::indicator {
                                        width: 30px;
                                        height: 30px;
                                    }
                                """)
        self.set_relative_geometry_3(self.C1_checkbox, 500, 250, 74, 50, base_width, base_height, current_width,
                                     current_height, fontsize=16, paddingsize=0)

        # ----- C2 Checkbox -----
        self.C2_checkbox = QtWidgets.QCheckBox(page)
        self.C2_checkbox.setText("C2")
        self.C2_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C2_checkbox.setStyleSheet("""
                                    QCheckBox {
                                        color: black;
                                        border-radius: 10px;
                                    }
                                    QCheckBox::indicator {
                                        width: 30px;
                                        height: 30px;
                                    }
                                """)
        self.set_relative_geometry_3(self.C2_checkbox, 600, 250, 74, 50, base_width, base_height, current_width,
                                     current_height, fontsize=16, paddingsize=0)

        # ----- C3 Checkbox -----
        self.C3_checkbox = QtWidgets.QCheckBox(page)
        self.C3_checkbox.setText("C3")
        self.C3_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C3_checkbox.setStyleSheet("""
                                    QCheckBox {
                                        color: black;
                                        border-radius: 10px;
                                    }
                                    QCheckBox::indicator {
                                        width: 30px;
                                        height: 30px;
                                    }
                                """)
        self.set_relative_geometry_3(self.C3_checkbox, 700, 250, 74, 50, base_width, base_height, current_width,
                                     current_height, fontsize=16, paddingsize=0)

        # ----- C4 Checkbox -----
        self.C4_checkbox = QtWidgets.QCheckBox(page)
        self.C4_checkbox.setText("C4")
        self.C4_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C4_checkbox.setStyleSheet("""
                                    QCheckBox {
                                        color: black;
                                        border-radius: 10px;
                                    }
                                    QCheckBox::indicator {
                                        width: 30px;
                                        height: 30px;
                                    }
                                """)
        self.set_relative_geometry_3(self.C4_checkbox, 800, 250, 74, 50, base_width, base_height, current_width,
                                     current_height, fontsize=16, paddingsize=0)

        # ----- C5 Checkbox -----
        self.C5_checkbox = QtWidgets.QCheckBox(page)
        self.C5_checkbox.setText("C5")
        self.C5_checkbox.stateChanged.connect(self.on_checkbox_changed)
        self.C5_checkbox.setStyleSheet("""
                                        QCheckBox {
                                            color: black;
                                            border-radius: 10px;
                                        }
                                        QCheckBox::indicator {
                                            width: 30px;
                                            height: 30px;
                                        }
                                    """)
        self.set_relative_geometry_3(self.C5_checkbox, 900, 250, 74, 50, base_width, base_height, current_width,
                                     current_height, fontsize=16, paddingsize=0)

        # Button Show Graph 3_1
        self.showGraph3_1 = QtWidgets.QPushButton(page)
        self.showGraph3_1.setText("Radar Chart")
        self.showGraph3_1.setObjectName("button_3_1")
        self.showGraph3_1.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)

        self.set_relative_geometry_3(self.showGraph3_1, 450, 350, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph3_1.clicked.connect(self.plot_3_1)

        # Button Show Graph 3_2
        self.showGraph3_2 = QtWidgets.QPushButton(page)
        self.showGraph3_2.setText("Feature Difference Bar chart")
        self.showGraph3_2.setObjectName("button_3_2")
        self.showGraph3_2.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph3_2, 750, 350, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=14)
        self.showGraph3_2.clicked.connect(self.plot_3_2)

        # Button Show Graph 3_3
        self.showGraph3_3 = QtWidgets.QPushButton(page)
        self.showGraph3_3.setText("Distribution Overlap Plot")
        self.showGraph3_3.setObjectName("button_3_3")
        self.showGraph3_3.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph3_3, 450, 450, 200, 50, base_width, base_height, current_width,
                                     current_height)
        self.showGraph3_3.clicked.connect(self.plot_3_3)

        # Button Show Graph 3_4
        self.showGraph3_4 = QtWidgets.QPushButton(page)
        self.showGraph3_4.setText("Distance Plot")
        self.showGraph3_4.setObjectName("button_3_4")
        self.showGraph3_4.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph3_4, 750, 450, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph3_4.clicked.connect(self.plot_3_4)

        # Label for graph 3
        self.graph_label3 = QtWidgets.QLabel(page)
        self.graph_label3.setStyleSheet("border: 0px solid black;")
        self.set_relative_geometry_3(self.graph_label3, 1080, 100, 800, 800, base_width, base_height, current_width, current_height)


    def go_to_page3(self):
        self.stacked_widget.setCurrentWidget(self.page3)
        self.set_relative_geometry_3(self.MP_combo, 550, 160, 230, 60, base_width, base_height, current_width,
                                     current_height, fontsize=22, paddingsize=8)

    def setupPage4(self, page):

        page.setObjectName("Page4")
        page.setStyleSheet("background-color: lightblue;")

        self.labelInfoCluster4 = QtWidgets.QLabel(page)
        self.labelInfoCluster4.setText(
            "Cluster 0: regular but more spending customers\n"
            "Cluster 1: largest group of regular customers\n"
            "Cluster 2: frequent orders but low loyality\n"
            "Cluster 3: high frequency, small orders\n"
            "Cluster 4: best spending customers\n"
            "Cluster 5: least spending customers")
        self.labelInfoCluster4.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         border-radius: 10px;
                                     }
                        """)
        self.set_relative_geometry_3(self.labelInfoCluster4, 50, 700, 500, 130, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        self.label4 = QtWidgets.QLabel(page)
        self.label4.setText("Insights in all clusters")
        self.label4.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         border-radius: 10px;
                                     }
                        """)
        self.set_relative_geometry_3(self.label4, 450, 50, 300, 30, base_width, base_height, current_width,
                                     current_height, fontsize=28)

        self.to_page1_button = QtWidgets.QPushButton(page)
        self.to_page1_button.setText("back")
        self.to_page1_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.to_page1_button.clicked.connect(self.go_to_page1)
        self.set_relative_geometry_3(self.to_page1_button, 50, 150, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ----- 4 Combo Box -----
        self.P4_combo = QComboBox(page)
        self.P4_combo.addItems(["--choose--"] + currentFeatures)
        self.P4_combo.setObjectName("P4_combo")
        self.P4_combo.setStyleSheet("""
                                                        QComboBox {
                                                                background-color: white;
                                                                color: black;
                                                                border: 2px solid white;
                                                                border-radius: 10px;
                                                            }
                                                        QComboBox:focus {
                                                            border: 2px solid blue;
                                                        }
                                                        QComboBox:editable:on {
                                                            border: 2px solid blue;
                                                        }
                                                        QComboBox:!editable:on {
                                                            border: 2px solid blue;
                                                        }
                                                        QComboBox QAbstractItemView {
                                                                    background-color: white;
                                                                    color: black;
                                                                    border: 2px solid white;
                                                                    selection-background-color: lightblue;
                                                                    selection-color: black;
                                                        }
                                                                    """)

        self.P4_combo.view().setMinimumWidth(200)
        self.P4_combo.view().setMinimumHeight(300)
        self.set_relative_geometry_3(self.P4_combo, 710, 620, 180, 40, base_width, base_height, current_width,
                                     current_height)
        self.P4_combo.currentIndexChanged.connect(lambda: self.on_combobox_change_2("P4_combo"))

        # Button Show Graph 4_1
        self.showGraph4_1 = QtWidgets.QPushButton(page)
        self.showGraph4_1.setText("Cluster Infos")
        self.showGraph4_1.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph4_1, 400, 260, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph4_1.clicked.connect(self.plot_4_1)

        # Button Show Graph 4_2
        self.showGraph4_2 = QtWidgets.QPushButton(page)
        self.showGraph4_2.setText("Cluster Sizes")
        self.showGraph4_2.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph4_2, 400, 360, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph4_2.clicked.connect(self.plot_4_2)

        # Button Show Graph 4_3
        self.showGraph4_3 = QtWidgets.QPushButton(page)
        self.showGraph4_3.setText("Cluster Overwiew")
        self.showGraph4_3.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph4_3, 400, 460, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph4_3.clicked.connect(self.plot_4_3)


        # Button Show Graph 4_4
        self.showGraph4_4 = QtWidgets.QPushButton(page)
        self.showGraph4_4.setText("Feature Importance p Cluster")
        self.showGraph4_4.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph4_4, 700, 460, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=14)
        self.showGraph4_4.clicked.connect(self.plot_4_4)

        # Button Show Graph 4_5
        self.showGraph4_5 = QtWidgets.QPushButton(page)
        self.showGraph4_5.setText("Radar Chart")
        self.showGraph4_5.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph4_5, 700, 260, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph4_5.clicked.connect(self.plot_4_5)

        # Button Show Graph 4_6
        self.showGraph4_6 = QtWidgets.QPushButton(page)
        self.showGraph4_6.setText("Distribution Overlap Plot")
        self.showGraph4_6.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph4_6, 700, 560, 200, 50, base_width, base_height, current_width,
                                     current_height)
        self.showGraph4_6.clicked.connect(self.plot_4_6)

        # Button Show Graph 4_7
        self.showGraph4_7 = QtWidgets.QPushButton(page)
        self.showGraph4_7.setText("Distance Plot")
        self.showGraph4_7.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph4_7, 700, 360, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph4_7.clicked.connect(self.plot_4_7)

        # Button Show Graph 4_8
        self.showGraph4_8 = QtWidgets.QPushButton(page)
        self.showGraph4_8.setText("3D Cluster PCA")
        self.showGraph4_8.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.showGraph4_8, 400, 560, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.showGraph4_8.clicked.connect(self.plot_4_8)

        #Loading label 3d
        self.load_label_3d = QtWidgets.QLabel(page)
        self.load_label_3d.setText("... Loading ...")
        self.load_label_3d.setAlignment(Qt.AlignCenter)
        self.set_relative_geometry_3(self.load_label_3d, 690, 870, 300, 80, base_width, base_height, current_width,
                                     current_height, fontsize=30, paddingsize=30)
        self.load_label_3d.setVisible(False)

        #Timer for animation
        self.timer3d = QTimer(page)
        self.timer3d.timeout.connect(self.animate_loading)
        self.timer3d.start(500)

        self.web_view = QWebEngineView(page)

        # Label for graph 4
        self.graph_label4 = QtWidgets.QLabel(page)
        self.graph_label4.setStyleSheet("border: 0px solid black;")
        self.set_relative_geometry_3(self.graph_label4, 1080, 100, 800, 800, base_width, base_height, current_width,
                                     current_height)



    def go_to_page4(self):
        self.stacked_widget.setCurrentWidget(self.page4)
        self.set_relative_geometry_3(self.MP_combo, 500, 160, 230, 60, base_width, base_height, current_width,
                                     current_height, fontsize=22, paddingsize=8)

    def setupPage5(self, page):
        page.setObjectName("Page5")
        page.setStyleSheet("background-color: lightblue;")


        # Loading label 3d
        self.load_label_5 = QtWidgets.QLabel(page)
        self.load_label_5.setText("... Loading ...")
        self.load_label_5.setAlignment(Qt.AlignCenter)
        self.load_label_5.setStyleSheet("border: 5px solid black;")
        self.set_relative_geometry_3(self.load_label_5, 1100, 450, 300, 80, base_width, base_height, current_width,
                                     current_height, fontsize=24)
        self.load_label_5.setVisible(False)


        self.label5 = QtWidgets.QLabel(page)
        self.label5.setText("Connect new entry to cluster")
        self.label5.setStyleSheet("""
                                    QLabel {
                                         background-color: lightgray;
                                         color: black;
                                         border: 2px solid white;
                                         border-radius: 10px;
                                     }
                        """)
        self.set_relative_geometry_3(self.label5, 450, 50, 300, 30, base_width, base_height, current_width,
                                     current_height, fontsize=28)

        self.to_page1_button = QtWidgets.QPushButton(page)
        self.to_page1_button.setText("back")
        self.to_page1_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.to_page1_button, 50, 50, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=22)
        self.to_page1_button.clicked.connect(self.go_to_page1)

        # Calculate Button
        self.calc_cluster_button = QtWidgets.QPushButton(page)
        self.calc_cluster_button.setText("Quick prediction\nfor cluster")
        self.calc_cluster_button.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 3px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 8px solid lightgrey;
                                            border-radius: 25px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.calc_cluster_button, 950, 700, 200, 50, base_width, base_height, current_width,
                                     current_height, fontsize=20)
        self.calc_cluster_button.clicked.connect(lambda: self.calc_cluster("calc1"))

        # Calculate Precise Button
        self.calc_precise_cluster_button = QtWidgets.QPushButton(page)
        self.calc_precise_cluster_button.setText("Calculate cluster")
        self.calc_precise_cluster_button.setStyleSheet("""
                                                QPushButton {
                                                    background: qlineargradient(
                                                            x1: 0, y1: 0, x2: 1, y2: 1,
                                                            stop: 0 cornflowerblue,
                                                            stop: 1 darkblue
                                                    );
                                                    border: 3px solid lightgrey;
                                                    border-radius: 25px;
                                                    color: white;
                                                   }
                                                QPushButton:pressed {
                                                    background: qlineargradient(
                                                            x1: 0, y1: 0, x2: 1, y2: 1,
                                                            stop: 0 cornflowerblue,
                                                            stop: 1 darkblue
                                                    );
                                                    border: 8px solid lightgrey;
                                                    border-radius: 25px;
                                                    color: white;
                                                }
                                                   """)
        self.set_relative_geometry_3(self.calc_precise_cluster_button, 1300, 700, 200, 50, base_width, base_height,
                                     current_width,
                                     current_height, fontsize=22)
        self.calc_precise_cluster_button.clicked.connect(lambda: self.calc_cluster("calc2"))

        # Label for explanation
        self.labelExp = QtWidgets.QLabel(page)
        self.labelExp.setText("by calculating ward distance\nto cluster centroids;\nnot very accurate")
        self.labelExp.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.labelExp, 950, 790, 300, 30, base_width, base_height, current_width,
                                     current_height, fontsize=18)


        # Label for explanation2
        self.labelExp2 = QtWidgets.QLabel(page)
        self.labelExp2.setText("by recalculating clusters;\nmore accurate\n(might take some minutes)")
        self.labelExp2.setStyleSheet("""
                                                    QLabel {
                                                        background-color: lightgray;
                                                        color: black;
                                                        border: 2px solid white;
                                                        border-radius: 10px;
                                                    }
                                                """)
        self.set_relative_geometry_3(self.labelExp2, 1300, 760, 300, 30, base_width, base_height, current_width,
                                     current_height, fontsize=18)

        # Label for explanation3
        self.labelExp3 = QtWidgets.QLabel(page)
        self.labelExp3.setText("for final clustering this might not work with every machine\ndue to lack of memory, "
                               "also might take 30 minutes or more.\nchoose different clustering approach from combobox on top\nfor faster result")
        self.labelExp3.setStyleSheet("""
                                        QLabel {
                                            background-color: lightgray;
                                            color: black;
                                            border: 2px solid red;
                                            border-radius: 10px;
                                        }
                                    """)
        self.set_relative_geometry_3(self.labelExp3, 1300, 850, 300, 30, base_width, base_height, current_width,
                                     current_height, fontsize=18)




        # ----- TT1 Label -----
        self.TT1_label = QtWidgets.QLabel(page)
        self.TT1_label.setText("Vendor Count:")
        self.TT1_label.setStyleSheet("""
                                    QLabel {
                                        background-color: lightgray;
                                        color: black;
                                        border: 2px solid white;
                                        border-radius: 10px;
                                    }
                                """)
        self.set_relative_geometry_3(self.TT1_label, 50, 150, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- TT1 Input field -----
        self.TT1_input = QtWidgets.QLineEdit(page)
        self.TT1_input.setObjectName("TT1_input")
        self.TT1_input.setPlaceholderText("0-27, mean:2.9")
        self.TT1_input.returnPressed.connect(lambda: self.TT1_Confirm_Btn())
        self.TT1_input.textChanged.connect(lambda: self.on_text_change_input("TT1_label", "TT1_input"))
        self.TT1_input.setStyleSheet("""
                                    QLineEdit {
                                        background-color: white;
                                        color: black;
                                        border: 2px solid white;
                                        border-radius: 10px;
                                    }
                                    QLineEdit:focus {
                                        border: 2px solid blue;
                                    }
                                """)
        self.set_relative_geometry_3(self.TT1_input, 50, 200, 140, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- The TT1 Confirm Button -----
        self.TT1_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT1_Confirm_Btn())
        self.TT1_Confirm.setText("Confirm")
        self.TT1_Confirm.setObjectName("btn_TT1_Confirm")
        self.TT1_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT1_Confirm, 50, 250, 130, 30, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ---- The TT1 Error Label
        self.TT1_err = QtWidgets.QLabel(page)
        self.TT1_err.setText("Only positive numbers")
        self.TT1_err.setVisible(False)
        self.TT1_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                border-radius: 10px;
                                            }
                                                """)
        self.set_relative_geometry_3(self.TT1_err, 50, 300, 230, 30, base_width, base_height, current_width,
                                     current_height)

        # ---- The TT1 Warn Label
        self.TT1_warn = QtWidgets.QLabel(page)
        self.TT1_warn.setText("Unsual Values, might\nlead to different results")
        self.TT1_warn.setVisible(False)
        self.TT1_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT1_warn, 50, 300, 180, 40, base_width, base_height, current_width,
                                     current_height, paddingsize=1)





        # ----- TT2 Label -----
        self.TT2_label = QtWidgets.QLabel(page)
        self.TT2_label.setText("Product Count:")
        self.TT2_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT2_label, 350, 150, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- TT2 Input field -----
        self.TT2_input = QtWidgets.QLineEdit(page)
        self.TT2_input.textChanged.connect(lambda: self.on_text_change_input("TT2_label", "TT2_input"))
        self.TT2_input.returnPressed.connect(lambda: self.TT2_Confirm_Btn())
        self.TT2_input.setObjectName("TT2_input")
        self.TT2_input.setPlaceholderText("1-54, mean:5")
        self.TT2_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT2_input, 350, 200, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- The TT2 Confirm Button -----
        self.TT2_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT2_Confirm_Btn())
        self.TT2_Confirm.setText("Confirm")
        self.TT2_Confirm.setObjectName("btn_TT2_Confirm")
        self.TT2_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT2_Confirm, 350, 250, 130, 30, base_width, base_height, current_width,
                                     current_height, fontsize=22)



        # ---- The TT2 Error Label
        self.TT2_err = QtWidgets.QLabel(page)
        self.TT2_err.setText("Only positive numbers > 0")
        self.TT2_err.setVisible(False)
        self.TT2_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT2_err, 350, 300, 230, 30, base_width, base_height, current_width,
                                     current_height)

        # ---- The TT2 Warn Label
        self.TT2_warn = QtWidgets.QLabel(page)
        self.TT2_warn.setText("Unsual Values, might\nlead to different results")
        self.TT2_warn.setVisible(False)
        self.TT2_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT2_warn, 350, 300, 180, 40, base_width, base_height, current_width,
                                     current_height, paddingsize=1)


        # ----- TT3 Label -----
        self.TT3_label = QtWidgets.QLabel(page)
        self.TT3_label.setText("is Chain:")
        self.TT3_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT3_label, 650, 150, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- TT3 Input field -----
        self.TT3_input = QtWidgets.QLineEdit(page)
        self.TT3_input.textChanged.connect(lambda: self.on_text_change_input("TT3_label", "TT3_input"))
        self.TT3_input.returnPressed.connect(lambda: self.TT3_Confirm_Btn())
        self.TT3_input.setObjectName("TT3_input")
        self.TT3_input.setPlaceholderText("0-34, mean:2.5")
        self.TT3_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT3_input, 650, 200, 140, 30, base_width, base_height, current_width,
                                     current_height)


        # ----- The TT3 Confirm Button -----
        self.TT3_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT3_Confirm_Btn())
        self.TT3_Confirm.setText("Confirm")
        self.TT3_Confirm.setObjectName("btn_TT3_Confirm")
        self.TT3_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT3_Confirm, 650, 250, 130, 30, base_width, base_height, current_width,
                                     current_height)


        # ---- The TT3 Error Label
        self.TT3_err = QtWidgets.QLabel(page)
        self.TT3_err.setText("Only positive numbers")
        self.TT3_err.setVisible(False)
        self.TT3_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT3_err, 650, 300, 230, 30, base_width, base_height, current_width,
                                     current_height)

        # ---- The TT3 Warn Label
        self.TT3_warn = QtWidgets.QLabel(page)
        self.TT3_warn.setText("Unsual Values, might\nlead to different results")
        self.TT3_warn.setVisible(False)
        self.TT3_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT3_warn, 650, 300, 180, 40, base_width, base_height, current_width,
                                     current_height, paddingsize=1)


        # ----- TT4 Label -----
        self.TT4_label = QtWidgets.QLabel(page)
        self.TT4_label.setText("first Order:")
        self.TT4_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT4_label, 350, 400, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- TT4 Input field -----
        self.TT4_input = QtWidgets.QLineEdit(page)
        self.TT4_input.textChanged.connect(lambda: self.on_text_change_input("TT4_label", "TT4_input"))
        self.TT4_input.returnPressed.connect(lambda: self.TT4_Confirm_Btn())
        self.TT4_input.setObjectName("TT4_input")
        self.TT4_input.setPlaceholderText("0-90, mean:28")
        self.TT4_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT4_input, 350, 450, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- The TT4 Confirm Button -----
        self.TT4_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT4_Confirm_Btn())
        self.TT4_Confirm.setText("Confirm")
        self.TT4_Confirm.setObjectName("btn_TT4_Confirm")
        self.TT4_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT4_Confirm, 350, 500, 130, 30, base_width, base_height, current_width,
                                     current_height, fontsize=22)


        # ---- The TT4 Error Label
        self.TT4_err = QtWidgets.QLabel(page)
        self.TT4_err.setText("Only positive numbers")
        self.TT4_err.setVisible(False)
        self.TT4_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT4_err, 350, 550, 230, 30, base_width, base_height, current_width,
                                     current_height)

        # ---- The TT4 Warn Label
        self.TT4_warn = QtWidgets.QLabel(page)
        self.TT4_warn.setText("Unsual Values, might\nlead to different results")
        self.TT4_warn.setVisible(False)
        self.TT4_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT4_warn, 350, 550, 180, 40, base_width, base_height, current_width,
                                     current_height, paddingsize=1)


        # ----- TT5 Label -----
        self.TT5_label = QtWidgets.QLabel(page)
        self.TT5_label.setText("last Order:")
        self.TT5_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT5_label, 650, 400, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- TT5 Input field -----
        self.TT5_input = QtWidgets.QLineEdit(page)
        self.TT5_input.textChanged.connect(lambda: self.on_text_change_input("TT5_label", "TT5_input"))
        self.TT5_input.returnPressed.connect(lambda: self.TT5_Confirm_Btn())
        self.TT5_input.setObjectName("TT5_input")
        self.TT5_input.setPlaceholderText("0-90, mean:63")
        self.TT5_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT5_input, 650, 450, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- The TT5 Confirm Button -----
        self.TT5_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT5_Confirm_Btn())
        self.TT5_Confirm.setText("Confirm")
        self.TT5_Confirm.setObjectName("btn_TT5_Confirm")
        self.TT5_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT5_Confirm, 650, 500, 130, 30, base_width, base_height, current_width,
                                     current_height, fontsize=22)



        # ---- The TT5 Error Label
        self.TT5_err = QtWidgets.QLabel(page)
        self.TT5_err.setText("Only positive numbers")
        self.TT5_err.setVisible(False)
        self.TT5_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT5_err, 650, 550, 230, 30, base_width, base_height, current_width,
                                     current_height)

        # ---- The TT5 Warn Label
        self.TT5_warn = QtWidgets.QLabel(page)
        self.TT5_warn.setText("Unsual Values, might\nlead to different results")
        self.TT5_warn.setVisible(False)
        self.TT5_warn.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid orange;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT5_warn, 650, 550, 180, 40, base_width, base_height, current_width,
                                     current_height, paddingsize=1)



        # ----- TT6 Label -----
        self.TT6_label = QtWidgets.QLabel(page)
        self.TT6_label.setText("Total expenses:")
        self.TT6_label.setStyleSheet("""
                                    QLabel {
                                        background-color: lightgray;
                                        color: black;
                                        border: 2px solid white;
                                        border-radius: 10px;
                                    }
                                """)
        self.set_relative_geometry_3(self.TT6_label, 230, 650, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- TT6 Input Slider -----
        self.TT6_input_slider = QtWidgets.QSlider(page)
        self.TT6_input_slider.setOrientation(QtCore.Qt.Horizontal)
        self.TT6_input_slider.setMinimum(0)
        self.TT6_input_slider.setMaximum(500)
        self.TT6_input_slider.setValue(0)
        self.TT6_input_slider.setObjectName("TT6_input_slider")
        self.TT6_input_slider.valueChanged.connect(self.update_label_slide_TT6)
        #joao
        self.TT6_input_slider.setStyleSheet("""
            QSlider::groove:horizontal {
                border: 2px solid #bbb;
                background: white;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::sub-page:horizontal {
                background: blue;
                border: 2px solid #777;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::add-page:horizontal {
                background: #ddd;
                border: 2px solid #777;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::handle:horizontal {
                background: #333;
                border: 2px solid #555;
                width: 18px;
                height: 18px;
                margin: -5px 0; /* handle is placed outside groove */
                border-radius: 9px;
            }
            QSlider::handle:horizontal:hover {
                background: #444;
            }
            QSlider::handle:focus {
                background-color: lightgrey;
            }
            QSlider::sub-page:horizontal:focus {
                background: blue;
                border: 2px solid lightgrey;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::add-page:horizontal:focus {
                background: #ddd;
                border: 2px solid lightgrey;
                height: 8px;
                border-radius: 4px;
            }
            QSlider::sub-page:horizontal:disabled {
                background: #bbb;
                border-color: #999;
            }
            QSlider::add-page:horizontal:disabled {
                background: #eee;
                border-color: #999;
            }
            QSlider::handle:horizontal:disabled {
                background: #eee;
                border: 2px solid #aaa;
            }
        """)
        self.set_relative_geometry_3(self.TT6_input_slider, 180, 700, 230, 30, base_width, base_height, current_width,
                                     current_height)

        self.TT6_slider_label = QtWidgets.QLabel(page)
        self.TT6_slider_label.setAlignment(QtCore.Qt.AlignCenter)
        self.TT6_slider_label.setText("0  -  mean:35")
        self.TT6_slider_label.setObjectName("TT6_slider_label")
        self.TT6_slider_label.setStyleSheet("""
            QLabel {
                color: black;
                background-color: white;
                border: 2px solid gray;
                border-radius: 5px;
            }
        """)
        self.set_relative_geometry_3(self.TT6_slider_label, 230, 735, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- The TT6 Confirm Button -----
        self.TT6_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT6_Confirm_Btn())
        self.TT6_Confirm.setText("Confirm")
        self.TT6_Confirm.setObjectName("btn_TT6_Confirm")
        self.TT6_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT6_Confirm, 230, 785, 130, 30, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ---- The TT6 Warn Label
        self.TT6_warn = QtWidgets.QLabel(page)
        self.TT6_warn.setText("Unsual Values, might\nlead to different results")
        self.TT6_warn.setVisible(False)
        self.TT6_warn.setStyleSheet("""
                                        QLabel {
                                            background-color: lightgray;
                                            color: black;
                                            border: 2px solid orange;
                                            border-radius: 10px;
                                        }
                                    """)
        self.set_relative_geometry_3(self.TT6_warn, 180, 850, 180, 40, base_width, base_height, current_width,
                                     current_height, paddingsize=1)

        # ----- TT7 Label -----
        self.TT7_label = QtWidgets.QLabel(page)
        self.TT7_label.setText("Culinary Variety:")
        self.TT7_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT7_label, 580, 650, 130, 30, base_width, base_height, current_width,
                                     current_height)


        # ----- TT7 Input Slider -----
        self.TT7_input_slider = QtWidgets.QSlider(page)
        self.TT7_input_slider.setOrientation(QtCore.Qt.Horizontal)
        self.TT7_input_slider.setMinimum(0)
        self.TT7_input_slider.setMaximum(100)
        self.TT7_input_slider.setValue(0)
        self.TT7_input_slider.setObjectName("TT7_input_slider")
        self.TT7_input_slider.valueChanged.connect(self.update_label_slide_TT7)
        self.TT7_input_slider.setStyleSheet("""
                    QSlider::groove:horizontal {
                        border: 2px solid #bbb;
                        background: white;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::sub-page:horizontal {
                        background: blue;
                        border: 2px solid #777;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::add-page:horizontal {
                        background: #ddd;
                        border: 2px solid #777;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::handle:horizontal {
                        background: #333;
                        border: 2px solid #555;
                        width: 18px;
                        height: 18px;
                        margin: -5px 0; /* handle is placed outside groove */
                        border-radius: 9px;
                    }
                    QSlider::handle:focus {
                        background-color: lightgrey;
                    }
                    QSlider::sub-page:horizontal:focus {
                        background: blue;
                        border: 2px solid lightgrey;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::add-page:horizontal:focus {
                        background: #ddd;
                        border: 2px solid lightgrey;
                        height: 8px;
                        border-radius: 4px;
                    }
                    QSlider::handle:horizontal:hover {
                        background: #444;
                    }
                    QSlider::sub-page:horizontal:disabled {
                        background: #bbb;
                        border-color: #999;
                    }
                    QSlider::add-page:horizontal:disabled {
                        background: #eee;
                        border-color: #999;
                    }
                    QSlider::handle:horizontal:disabled {
                        background: #eee;
                        border: 2px solid #aaa;
                    }
                """)
        self.set_relative_geometry_3(self.TT7_input_slider, 530, 700, 230, 30, base_width, base_height, current_width,
                                     current_height)

        self.TT7_slider_label = QtWidgets.QLabel(page)
        self.TT7_slider_label.setAlignment(QtCore.Qt.AlignCenter)
        self.TT7_slider_label.setText("0  -  mean:0.14")
        self.TT7_slider_label.setObjectName("TT7_slider_label")
        self.TT7_slider_label.setStyleSheet("""
                    QLabel {
                        color: black;
                        background-color: white;
                        border: 2px solid gray;
                        border-radius: 5px;
                    }
                """)
        self.set_relative_geometry_3(self.TT7_slider_label, 580, 735, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- The TT7 Confirm Button -----
        self.TT7_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT7_Confirm_Btn())
        self.TT7_Confirm.setText("Confirm")
        self.TT7_Confirm.setObjectName("btn_TT7_Confirm")
        self.TT7_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT7_Confirm, 580, 785, 130, 30, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ----- TT8 Label -----
        self.TT8_label = QtWidgets.QLabel(page)
        self.TT8_label.setText("customer Age:")
        self.TT8_label.setStyleSheet("""
                                    QLabel {
                                        background-color: lightgray;
                                        color: black;
                                        border: 2px solid white;
                                        border-radius: 10px;
                                    }
                                """)
        self.set_relative_geometry_3(self.TT8_label, 50, 400, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- TT8 Combo Box -----
        self.TT8_combo = QComboBox(page)
        self.TT8_combo.addItem("---Choose---")
        self.TT8_combo.addItem("15-17")
        self.TT8_combo.addItem("18-22")
        self.TT8_combo.addItem("23-28")
        self.TT8_combo.addItem("29-35")
        self.TT8_combo.addItem("36-49")
        self.TT8_combo.addItem("50+")
        self.TT8_combo.setObjectName("TT8_combo")
        self.TT8_combo.setStyleSheet("""
                        QComboBox {
                                background-color: white;
                                color: black;
                                border: 2px solid white;
                                border-radius: 10px;
                            }
                        QComboBox:focus {
                            border: 2px solid blue;
                        }
                        QComboBox:editable:on {
                            border: 2px solid blue;
                        }
                        QComboBox:!editable:on {
                            border: 2px solid blue;
                        }
                        QComboBox QAbstractItemView {
                                    background-color: white;
                                    color: black;
                                    border: 2px solid white;
                                    selection-background-color: lightblue;
                                    selection-color: black;
                        }
                                    """)
        self.set_relative_geometry_3(self.TT8_combo, 50, 450, 130, 30, base_width, base_height, current_width,
                                     current_height)

        self.TT8_combo.view().setMinimumWidth(120)
        self.TT8_combo.currentIndexChanged.connect(lambda: self.on_combobox_change("TT8_combo", "TT8_err"))

        # ----- The TT8 Confirm Button -----
        self.TT8_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT8_Confirm_Btn())
        self.TT8_Confirm.setText("Confirm")
        self.TT8_Confirm.setObjectName("btn_TT8_Confirm")
        self.TT8_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT8_Confirm, 50, 500, 130, 30, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ---- The TT8 Error Label
        self.TT8_err = QtWidgets.QLabel(page)
        self.TT8_err.setText("Please Choose one")
        self.TT8_err.setVisible(False)
        self.TT8_err.setStyleSheet("""
                                                    QLabel {
                                                        background-color: lightgray;
                                                        color: black;
                                                        border: 2px solid red;
                                                        border-radius: 10px;
                                                    }
                                                """)
        self.set_relative_geometry_3(self.TT8_err, 50, 550, 180, 40, base_width, base_height, current_width,
                                     current_height)

        # ----- TT9 Label -----
        self.TT9_label = QtWidgets.QLabel(page)
        self.TT9_label.setText("Total Number\nof Orders placed:")
        self.TT9_label.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT9_label, 950, 150, 130, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- TT9 Input field -----
        self.TT9_input = QtWidgets.QLineEdit(page)
        self.TT9_input.textChanged.connect(lambda: self.on_text_change_input("TT9_label", "TT9_input"))
        self.TT9_input.returnPressed.connect(lambda: self.TT9_Confirm_Btn())
        self.TT9_input.setObjectName("TT9_input")
        self.TT9_input.setPlaceholderText("1-94, mean:4.3")
        self.TT9_input.setStyleSheet("""
                                            QLineEdit {
                                                background-color: white;
                                                color: black;
                                                border: 2px solid white;
                                                border-radius: 10px;
                                            }
                                            QLineEdit:focus {
                                                border: 2px solid blue;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT9_input, 950, 200, 140, 30, base_width, base_height, current_width,
                                     current_height)

        # ----- The TT9 Confirm Button -----
        self.TT9_Confirm = QtWidgets.QPushButton(page, clicked=lambda: self.TT9_Confirm_Btn())
        self.TT9_Confirm.setText("Confirm")
        self.TT9_Confirm.setObjectName("btn_TT9_Confirm")
        self.TT9_Confirm.setStyleSheet("""
                                        QPushButton {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 1px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                           }
                                        QPushButton:pressed {
                                            background: qlineargradient(
                                                    x1: 0, y1: 0, x2: 1, y2: 1,
                                                    stop: 0 cornflowerblue,
                                                    stop: 1 darkblue
                                            );
                                            border: 4px solid lightgrey;
                                            border-radius: 10px;
                                            color: white;
                                        }
                                           """)
        self.set_relative_geometry_3(self.TT9_Confirm, 950, 250, 130, 30, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ---- The TT9 Error Label
        self.TT9_err = QtWidgets.QLabel(page)
        self.TT9_err.setText("Only positive numbers > 0")
        self.TT9_err.setVisible(False)
        self.TT9_err.setStyleSheet("""
                                            QLabel {
                                                background-color: lightgray;
                                                color: black;
                                                border: 2px solid red;
                                                border-radius: 10px;
                                            }
                                        """)
        self.set_relative_geometry_3(self.TT9_err, 950, 300, 230, 30, base_width, base_height, current_width,
                                     current_height)

        # ---- The TT9 Warn Label
        self.TT9_warn = QtWidgets.QLabel(page)
        self.TT9_warn.setText("Unsual Values, might\nlead to different results")
        self.TT9_warn.setVisible(False)
        self.TT9_warn.setStyleSheet("""
                                                QLabel {
                                                    background-color: lightgray;
                                                    color: black;
                                                    border: 2px solid orange;
                                                    border-radius: 10px;
                                                }
                                            """)
        self.set_relative_geometry_3(self.TT9_warn, 950, 300, 180, 40, base_width, base_height, current_width,
                                     current_height, paddingsize=1)



        # ----- Prediction Label -----
        self.pred_label = QtWidgets.QLabel(page)
        self.pred_label.setText("No Prediction made yet")
        self.pred_label.setVisible(False)
        self.pred_label.setStyleSheet("""
                                                QLabel {
                                                    background-color: lightgray;
                                                    color: black;
                                                    border: 2px solid white;
                                                    border-radius: 10px;
                                                }

                                            """)
        self.set_relative_geometry_3(self.pred_label, 1100, 450, 400, 380, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ---- The error Label for full cluster calculation all features
        self.cluster_calc_err = QtWidgets.QLabel(page)
        self.cluster_calc_err.setText("Not available for Final Clustering,\ntoo much memory\n\n"
                                      "available only for perspective clusters")
        self.cluster_calc_err.setVisible(False)
        self.cluster_calc_err.setStyleSheet("""
                                                           QLabel {
                                                               background-color: lightgrey;
                                                               color: black;
                                                               border: 6px solid;
                                                               border-color: red;
                                                               border-radius: 10px;
                                                           }
                                                       """)
        self.set_relative_geometry_3(self.cluster_calc_err, 1300, 500, 250, 130, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ---- The NotConfirmed Error Label
        self.NotConfirmed_err = QtWidgets.QLabel(page)
        self.NotConfirmed_err.setText("Some entries are not confirmed")
        self.NotConfirmed_err.setVisible(False)
        self.NotConfirmed_err.setStyleSheet("""
                                                   QLabel {
                                                       background-color: darkblue;
                                                       color: white;
                                                       border: 6px solid;
                                                       border-color: darkblue;
                                                       border-radius: 10px;
                                                   }
                                               """)
        self.set_relative_geometry_3(self.NotConfirmed_err, 1100, 450, 250, 130, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ---- The First Order > Last Order Error Label
        self.FirstLast_err = QtWidgets.QLabel(page)
        self.FirstLast_err.setText("First Order cant be bigger than last order")
        self.FirstLast_err.setVisible(False)
        self.FirstLast_err.setStyleSheet("""
                                           QLabel {
                                               background-color: darkblue;
                                               color: white;
                                               border: 6px solid;
                                               border-color: red;
                                               border-radius: 10px;
                                           }
                                       """)
        self.set_relative_geometry_3(self.FirstLast_err, 1100, 450, 250, 130, base_width, base_height, current_width,
                                     current_height, fontsize=22)

        # ---- The Unsual Warn Label
        self.UnsualWarn = QtWidgets.QLabel(page)
        self.UnsualWarn.setText("Some entries have unsual values,\nresults may be unexcpected")
        self.UnsualWarn.setVisible(False)
        self.UnsualWarn.setStyleSheet("""
                                               QLabel {
                                                   background-color: lightgray;
                                                   color: black;
                                                   border: 6px solid orange;
                                                   border-radius: 10px;
                                               }
                                           """)
        self.set_relative_geometry_3(self.UnsualWarn, 1100, 525, 250, 130, base_width, base_height, current_width,
                                     current_height, fontsize=22)


        # ---- Info Calculate Label
        self.calc_info_label = QtWidgets.QLabel(page)
        self.calc_info_label.setText(
            "INFO: Other values will be calculated based on these:\n\nlifetime_days = last_order - first_order\n"
            "avg_per_product = total_expenses / product_count\navg_per_order = total_expenses / OrdersPlaced\n"
            "avg_order_size = product_count / OrdersPlaced\nchain_preference = is_chain / OrdersPlaced\n"
            "loyalty_to_venders = vendor_count / OrdersPlaced")
        self.calc_info_label.setStyleSheet("""
                                               QLabel {
                                                   background-color: lightgray;
                                                   color: black;
                                                   border: 2px solid black;
                                                   text-align: center;
                                                   border-radius: 10px;
                                               }
                                           """)
        self.set_relative_geometry_3(self.calc_info_label, 1350, 150, 160, 50, base_width, base_height, current_width,
                                     current_height, fontsize=18)


    def go_to_page5(self):
        self.stacked_widget.setCurrentWidget(self.page5)
        self.set_relative_geometry_3(self.MP_combo, 950, 50, 230, 60, base_width, base_height, current_width,
                                     current_height, fontsize=22, paddingsize=8)

    # -------------------------------------------------------------------------------------------------------------------

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("Interface", "Interface"))

In [5]:
def runMain():  # method to run the interface
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()

    sys.exit(app.exec_())

In [ ]:
runMain()  # run the interface